In [65]:
# Check Categorical Features in Dataset
print("CATEGORICAL FEATURES ANALYSIS")
print("=" * 60)

# List of categorical features to check
categorical_cols_to_check = [
    'business_sector',
    'entity_type', 
    'business_location',
    'capital_source'
]

print("Examining actual categorical values in the dataset...")
print("=" * 60)

for col in categorical_cols_to_check:
    if col in df.columns:
        unique_values = df[col].unique()
        print(f"\n{col.upper()}:")
        print(f"  Total unique values: {len(unique_values)}")
        
        # Show first 10 values to avoid too much output
        print(f"  Sample values (first 10): {list(unique_values[:10])}")
        
        # Show distribution of top 5
        value_counts = df[col].value_counts()
        print(f"  Top 5 most common:")
        for i, (val, count) in enumerate(value_counts.head().items(), 1):
            percentage = (count / len(df)) * 100
            print(f"    {i}. {val}: {count:,} ({percentage:.1f}%)")
    else:
        print(f"\n{col.upper()}: NOT FOUND in dataset")

print("\n" + "=" * 60)
print("Ready to create realistic sample predictions with actual data values!")

CATEGORICAL FEATURES ANALYSIS
Examining actual categorical values in the dataset...

BUSINESS_SECTOR:
  Total unique values: 24
  Sample values (first 10): ['Transportation And Storage', 'Financial And Insurance Activities', 'Wholesale And Retail Trade; Repair Of Motor Vehicles And Motorcycles', 'Other Service Activities', 'Construction', 'Mining And Quarrying', 'Accommodation And Food Service Activities', 'Activities Of Households As Employers; Undifferentiated Goods- And Services-Producing Activities Of Households For Own Use', 'Motorcycle transport', 'Manufacturing']
  Top 5 most common:
    1. Other Service Activities: 3,209 (21.4%)
    2. Wholesale And Retail Trade; Repair Of Motor Vehicles And Motorcycles: 2,559 (17.1%)
    3. Transportation And Storage: 1,193 (8.0%)
    4. Financial And Insurance Activities: 890 (5.9%)
    5. Accommodation And Food Service Activities: 726 (4.8%)

ENTITY_TYPE:
  Total unique values: 7
  Sample values (first 10): ['INDIVIDUAL', 'PRIVATE CORPORATIO

## Existing Business Success Predictor - Rwanda

### Project Overview
This notebook helps predict the success or failure of existing SMEs in Rwanda using operational data and performance metrics. Unlike pre-investment prediction, this model analyzes businesses with actual operational history to assess their likelihood of continued success.

#### What We'll Do:
- Analyze existing SME operational data and revenue trends
- Build a prediction model using financial performance indicators
- Create a business health assessment system
- Provide actionable insights for business improvement

#### Key Focus Areas:
- Multi-year revenue analysis (2021-2024)
- Growth trajectory assessment
- Operational efficiency metrics
- Business resilience indicators

Let's help assess and improve existing Rwandan SME performance! 🇷🇼

### 1. Import Libraries & Setup

In [27]:
# ===================================================================
# IMPORT REQUIRED LIBRARIES
# ===================================================================

# Core data processing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys
from datetime import datetime

# Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.feature_selection import SelectKBest, f_classif, RFE, SelectFromModel
import xgboost as xgb
import joblib

# Visualization libraries
plt.style.use('default')
sns.set_palette("husl")

print("All libraries imported successfully!")
print("Ready to analyze SME data for Rwanda")

All libraries imported successfully!
Ready to analyze SME data for Rwanda


In [11]:
# ===================================================================
# DATA LOADING AND INITIAL SETUP
# ===================================================================

import warnings
warnings.filterwarnings('ignore')

# File paths and data source configuration
DATA_PATH = '../data/sme_final_15k_enhanced.csv'  # Updated to new enhanced dataset
MODELS_DIR = '../models'

print("EXISTING BUSINESS SUCCESS PREDICTOR")
print("=" * 60)
print(f" Data source: {DATA_PATH}")
print(f" Models directory: {MODELS_DIR}")
print(f" Analysis focus: Existing business future success prediction")
print(f" Dataset: Enhanced with employment tracking (15K businesses)")
print("=" * 60)

# Enhanced feature configuration for new dataset
EXISTING_BUSINESS_FEATURES = [
    # Business fundamentals
    'business_capital',
    'number_of_employees',
    'business_sector',
    'entity_type',
    'owner_age',
    'owner_business_experience',
    
    # Revenue indicators (historical: 2021-2023)
    'turnover_first_year',   # 2021 data
    'turnover_second_year',  # 2022 data  
    'turnover_third_year',   # 2023 data
    'turnover_growth',
    
    # Employment indicators (NEW - enhanced dataset)
    'employment_first_year',   # 2021 employment
    'employment_second_year',  # 2022 employment
    'employment_third_year',   # 2023 employment 
    'employment_growth',
    
    # Engineered features (to be calculated)
    'revenue_growth_rate',
    'revenue_consistency_score',
    'revenue_per_employee_trend',
    'employment_efficiency',
    'business_scaling_indicator'
]

print(f"Enhanced feature set planned: {len(EXISTING_BUSINESS_FEATURES)} variables")
print(f"New employment tracking features included")
print(f"Revenue + Employment dual analysis approach")
print("=" * 60)

EXISTING BUSINESS SUCCESS PREDICTOR
 Data source: ../data/sme_final_15k_enhanced.csv
 Models directory: ../models
 Analysis focus: Existing business future success prediction
 Dataset: Enhanced with employment tracking (15K businesses)
Enhanced feature set planned: 19 variables
New employment tracking features included
Revenue + Employment dual analysis approach


In [12]:
# ===================================================================
# LOAD ENHANCED DATASET
# ===================================================================

# Load the enhanced dataset with employment tracking
try:
    df = pd.read_csv(DATA_PATH)
    print("DATASET LOADING SUCCESS")
    print("=" * 50)
    print(f"Dataset shape: {df.shape[0]:,} businesses × {df.shape[1]} features")
    print(f"File size: {os.path.getsize(DATA_PATH) / (1024*1024):.2f} MB")
    print(f"Target variable: business_status (Success/Fail)")
    
    # Display dataset info
    print("\nDATASET OVERVIEW:")
    print("-" * 30)
    print(f"• Total businesses: {df.shape[0]:,}")
    print(f"• Features available: {df.shape[1]}")
    print(f"• Memory usage: {df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")
    
    # Check for new employment features
    employment_cols = [col for col in df.columns if 'employment' in col.lower()]
    revenue_cols = [col for col in df.columns if 'turnover' in col.lower()]
    
    print(f"\nENHANCED FEATURES DETECTED:")
    print(f"• Revenue tracking columns: {len(revenue_cols)}")
    print(f"• Employment tracking columns: {len(employment_cols)}")
    print(f"• Employment features: {employment_cols}")
    
    # Quick data quality check
    missing_data = df.isnull().sum().sum()
    print(f"\nDATA QUALITY:")
    print(f"• Missing values: {missing_data:,}")
    print(f"• Data completeness: {((df.size - missing_data) / df.size * 100):.2f}%")
    
    print("=" * 50)
    
except FileNotFoundError:
    print("ERROR: Dataset file not found!")
    print(f"Expected location: {DATA_PATH}")
except Exception as e:
    print(f"ERROR loading dataset: {str(e)}")
    df = None

DATASET LOADING SUCCESS
Dataset shape: 15,000 businesses × 22 features
File size: 3.17 MB
Target variable: business_status (Success/Fail)

DATASET OVERVIEW:
------------------------------
• Total businesses: 15,000
• Features available: 22
• Memory usage: 9.35 MB

ENHANCED FEATURES DETECTED:
• Revenue tracking columns: 5
• Employment tracking columns: 5
• Employment features: ['employment_2021', 'employment_2022', 'employment_2023', 'employment_2024', 'employment_growth']

DATA QUALITY:
• Missing values: 75
• Data completeness: 99.98%


## 2. Data Loading & Initial Exploration

Let's load our SME dataset and see what we're working with!

In [13]:
# ===================================================================
# FIELD MAPPING: ABSOLUTE TO RELATIVE YEARS
# ===================================================================

print("FIELD MAPPING AND DATA PREPARATION")
print("=" * 50)

# Create field mapping from absolute years to relative years
# This aligns with the frontend form fields and provides flexibility

field_mapping = {
    # Revenue mapping (2021-2024 → first_year to fourth_year)
    'turnover_2021': 'turnover_first_year',
    'turnover_2022': 'turnover_second_year', 
    'turnover_2023': 'turnover_third_year',
    'turnover_2024': 'turnover_fourth_year',
    
    # Employment mapping (2021-2024 → first_year to fourth_year)
    'employment_2021': 'employment_first_year',
    'employment_2022': 'employment_second_year',
    'employment_2023': 'employment_third_year', 
    'employment_2024': 'employment_fourth_year'
}

# Apply field mapping
print("Applying relative year field mapping...")
for old_name, new_name in field_mapping.items():
    if old_name in df.columns:
        df[new_name] = df[old_name]
        print(f"   {old_name} → {new_name}")

print(f"\nField mapping complete!")
print(f"Dataset now uses relative year naming")
print("=" * 50)

FIELD MAPPING AND DATA PREPARATION
Applying relative year field mapping...
   turnover_2021 → turnover_first_year
   turnover_2022 → turnover_second_year
   turnover_2023 → turnover_third_year
   turnover_2024 → turnover_fourth_year
   employment_2021 → employment_first_year
   employment_2022 → employment_second_year
   employment_2023 → employment_third_year
   employment_2024 → employment_fourth_year

Field mapping complete!
Dataset now uses relative year naming


In [14]:
# ===================================================================
# ENHANCED FEATURE ENGINEERING
# ===================================================================

print("ENHANCED FEATURE ENGINEERING")
print("=" * 50)

# Calculate revenue-based features
print("Calculating revenue indicators...")

# Revenue growth rate (3-year progression: first_year to third_year)
def calculate_revenue_growth_rate(row):
    first_year = row['turnover_first_year']
    third_year = row['turnover_third_year']
    
    if first_year == 0:
        return 300 if third_year > 0 else 0  # Cap for businesses starting with 0
    else:
        return ((third_year - first_year) / first_year) * 100

df['revenue_growth_rate'] = df.apply(calculate_revenue_growth_rate, axis=1)

# Revenue consistency score (lower volatility = higher consistency)
revenue_cols = ['turnover_first_year', 'turnover_second_year', 'turnover_third_year']
df['revenue_std'] = df[revenue_cols].std(axis=1)
df['revenue_mean'] = df[revenue_cols].mean(axis=1)
df['revenue_consistency_score'] = 1 / (1 + (df['revenue_std'] / (df['revenue_mean'] + 1)))

# Revenue per employee trend (using third year data)
df['revenue_per_employee_current'] = df['turnover_third_year'] / (df['employment_third_year'] + 1)
df['revenue_per_employee_initial'] = df['turnover_first_year'] / (df['employment_first_year'] + 1)
df['revenue_per_employee_trend'] = df['revenue_per_employee_current'] - df['revenue_per_employee_initial']

print("   Revenue growth rate calculated")
print("   Revenue consistency score calculated") 
print("   Revenue per employee trend calculated")

# Calculate employment-based features (NEW!)
print("Calculating employment indicators...")

# Employment efficiency (revenue per employee improvement)
df['employment_efficiency'] = df['revenue_per_employee_current'] / (df['revenue_per_employee_initial'] + 1)

# Business scaling indicator (combined revenue + employment growth)
def calculate_scaling_indicator(row):
    revenue_growth = row['revenue_growth_rate']
    employment_growth_map = {'Increased': 1, 'Stable': 0, 'Decreased': -1}
    employment_score = employment_growth_map.get(row['employment_growth'], 0)
    
    # Scaling score: positive revenue growth + employment growth
    if revenue_growth > 10 and employment_score >= 0:
        return 'High_Scaling'
    elif revenue_growth > 0 and employment_score >= 0:
        return 'Moderate_Scaling'
    elif revenue_growth <= 0 and employment_score < 0:
        return 'Declining'
    else:
        return 'Mixed_Performance'

df['business_scaling_indicator'] = df.apply(calculate_scaling_indicator, axis=1)

print("   Employment efficiency calculated")
print("   Business scaling indicator calculated")

# Summary of new features
new_features = [
    'revenue_growth_rate', 'revenue_consistency_score', 
    'revenue_per_employee_trend', 'employment_efficiency',
    'business_scaling_indicator'
]

print(f"\nFeature engineering complete!")
print(f"Added {len(new_features)} new engineered features")
print(f"Ready for model training with enhanced dataset")
print("=" * 50)

ENHANCED FEATURE ENGINEERING
Calculating revenue indicators...
   Revenue growth rate calculated
   Revenue consistency score calculated
   Revenue per employee trend calculated
Calculating employment indicators...
   Employment efficiency calculated
   Business scaling indicator calculated

Feature engineering complete!
Added 5 new engineered features
Ready for model training with enhanced dataset
   Employment efficiency calculated
   Business scaling indicator calculated

Feature engineering complete!
Added 5 new engineered features
Ready for model training with enhanced dataset


In [15]:
# ===================================================================
# PHASE 1 COMPLETION SUMMARY
# ===================================================================

print("PHASE 1: DATA PREPARATION & SETUP - COMPLETE!")
print("=" * 60)

# Display final dataset information
print(f"FINAL DATASET OVERVIEW:")
print(f"   • Total businesses: {df.shape[0]:,}")
print(f"   • Total features: {df.shape[1]}")
print(f"   • Target variable: business_status")

# Show success distribution
success_dist = df['business_status'].value_counts()
success_pct = df['business_status'].value_counts(normalize=True) * 100
print(f"\nSUCCESS DISTRIBUTION:")
for status, count in success_dist.items():
    percentage = success_pct[status]
    print(f"   • {status}: {count:,} businesses ({percentage:.1f}%)")

# Show new engineered features
new_features = [
    'revenue_growth_rate', 'revenue_consistency_score', 
    'revenue_per_employee_trend', 'employment_efficiency',
    'business_scaling_indicator'
]

print(f"\nNEW ENGINEERED FEATURES:")
for feature in new_features:
    print(f"   {feature}")

# Show field mapping results
relative_year_fields = [col for col in df.columns if 'first_year' in col or 'second_year' in col or 'third_year' in col or 'fourth_year' in col]
print(f"\nRELATIVE YEAR FIELDS AVAILABLE:")
for field in relative_year_fields:
    print(f"   • {field}")

print(f"\nREADY FOR NEXT PHASES:")
print(f"   COMPLETE: Phase 1: Data Preparation & Setup")
print(f"   NEXT: Phase 2: Exploratory Data Analysis (EDA)")
print(f"   NEXT: Phase 3: Feature Selection") 
print(f"   NEXT: Phase 4: Model Development (3 Algorithms)")
print(f"   NEXT: Phase 5: Model Evaluation & Comparison")
print(f"   NEXT: Phase 6: Model Interpretation & Insights")
print(f"   NEXT: Phase 7: Recommendation System")

print("=" * 60)

PHASE 1: DATA PREPARATION & SETUP - COMPLETE!
FINAL DATASET OVERVIEW:
   • Total businesses: 15,000
   • Total features: 39
   • Target variable: business_status

SUCCESS DISTRIBUTION:
   • Success: 9,000 businesses (60.0%)
   • Fail: 6,000 businesses (40.0%)

NEW ENGINEERED FEATURES:
   revenue_growth_rate
   revenue_consistency_score
   revenue_per_employee_trend
   employment_efficiency
   business_scaling_indicator

RELATIVE YEAR FIELDS AVAILABLE:
   • turnover_first_year
   • turnover_second_year
   • turnover_third_year
   • turnover_fourth_year
   • employment_first_year
   • employment_second_year
   • employment_third_year
   • employment_fourth_year

READY FOR NEXT PHASES:
   COMPLETE: Phase 1: Data Preparation & Setup
   NEXT: Phase 2: Exploratory Data Analysis (EDA)
   NEXT: Phase 3: Feature Selection
   NEXT: Phase 4: Model Development (3 Algorithms)
   NEXT: Phase 5: Model Evaluation & Comparison
   NEXT: Phase 6: Model Interpretation & Insights
   NEXT: Phase 7: Recommen

In [ ]:
# Business Success vs Revenue Analysis (Updated for Relative Years)
print("BUSINESS SUCCESS vs REVENUE PATTERNS")
print("=" * 50)

# Check for business_status column
if 'business_status' in df.columns:
    print("Analyzing revenue patterns by business success status...")
    
    # Group by business status
    success_groups = df.groupby('business_status')
    
    print(f"\nBUSINESS STATUS DISTRIBUTION:")
    status_counts = df['business_status'].value_counts()
    for status, count in status_counts.items():
        percentage = (count / len(df)) * 100
        print(f"   {status}: {count:,} businesses ({percentage:.1f}%)")
    
    # Revenue statistics by success status (using new column names)
    print(f"\nREVENUE COMPARISON BY SUCCESS STATUS:")
    print("-" * 40)
    
    for status in df['business_status'].unique():
        if pd.notna(status):
            subset = df[df['business_status'] == status]
            print(f"\n{status.upper()} BUSINESSES:")
            
            # Calculate averages using new column names
            first_year_avg = subset['turnover_first_year'].mean()
            fourth_year_avg = subset['turnover_fourth_year'].mean()
            
            print(f"   Average First Year Revenue: {first_year_avg:,.0f} RWF")
            print(f"   Average Fourth Year Revenue: {fourth_year_avg:,.0f} RWF")
            
            if first_year_avg > 0:
                growth = ((fourth_year_avg - first_year_avg) / first_year_avg) * 100
                print(f"   Average 4-Year Growth: {growth:.1f}%")
            
            # Growth pattern distribution
            if 'turnover_growth' in subset.columns:
                growth_dist = subset['turnover_growth'].value_counts()
                print(f"   Growth Patterns:")
                for pattern, count in growth_dist.items():
                    pct = (count / len(subset)) * 100
                    print(f"     {pattern}: {pct:.1f}%")

else:
    print("  business_status column not found in dataset")
    print("   Cannot perform success vs revenue analysis")

print("\n" + "=" * 50)

BUSINESS SUCCESS vs REVENUE PATTERNS
Analyzing revenue patterns by business success status...

BUSINESS STATUS DISTRIBUTION:
   Success: 9,000 businesses (60.0%)
   Fail: 6,000 businesses (40.0%)

REVENUE COMPARISON BY SUCCESS STATUS:
----------------------------------------

SUCCESS BUSINESSES:
   Average First Year Revenue: 91,437,766 RWF
   Average Fourth Year Revenue: 202,396,549 RWF
   Average 4-Year Growth: 121.3%
   Growth Patterns:
     Increased: 71.6%
     Decreased: 27.0%
     Stable: 1.4%

FAIL BUSINESSES:
   Average First Year Revenue: 3,073,240 RWF
   Average Fourth Year Revenue: 4,646,779 RWF
   Average 4-Year Growth: 51.2%
   Growth Patterns:
     Decreased: 49.4%
     Increased: 39.8%
     Stable: 10.9%



# Phase 2: Exploratory Data Analysis

Comprehensive analysis of business success patterns and key performance indicators.

## 2.1 Business Success Distribution Analysis

In [31]:
# Business Success Distribution Analysis
print("Business Success Distribution Analysis")
print("=" * 50)

# Overall success distribution
success_distribution = df['business_success'].value_counts()
success_percentage = df['business_success'].value_counts(normalize=True) * 100

print("\nOverall Business Success Distribution:")
for status, count in success_distribution.items():
    percentage = success_percentage[status]
    status_label = "Success" if status == 1 else "Failure"
    print(f"{status_label}: {count:,} businesses ({percentage:.1f}%)")

# Success by business entity type
print("\nSuccess Rate by Business Entity:")
entity_success_rates = df.groupby('entity_type')['business_success'].agg(['count', 'mean']).round(3)
entity_success_rates.columns = ['Total_Businesses', 'Success_Rate']
entity_success_rates = entity_success_rates.sort_values('Success_Rate', ascending=False)

for entity, row in entity_success_rates.iterrows():
    print(f"{entity}: {row['Success_Rate']:.3f} ({row['Total_Businesses']:,} businesses)")

# Success by sector
print("\nSuccess Rate by Business Sector:")
sector_success_rates = df.groupby('business_sector')['business_success'].agg(['count', 'mean']).round(3)
sector_success_rates.columns = ['Total_Businesses', 'Success_Rate']
sector_success_rates = sector_success_rates.sort_values('Success_Rate', ascending=False)

# Show top 10 sectors by success rate
print("Top 10 Sectors by Success Rate:")
for sector, row in sector_success_rates.head(10).iterrows():
    print(f"{sector}: {row['Success_Rate']:.3f} ({row['Total_Businesses']:,} businesses)")

Business Success Distribution Analysis

Overall Business Success Distribution:
Success: 9,000 businesses (60.0%)
Failure: 6,000 businesses (40.0%)

Success Rate by Business Entity:
JOINT VENTURE: 0.749 (1,596.0 businesses)
PRIVATE CORPORATION: 0.741 (4,247.0 businesses)
COOPERATIVE: 0.736 (1,760.0 businesses)
LIMITED LIABILITY COMPANY: 0.638 (1,036.0 businesses)
SOLE PROPRIETORSHIP: 0.569 (65.0 businesses)
PARTNERSHIP: 0.543 (554.0 businesses)
INDIVIDUAL: 0.412 (5,742.0 businesses)

Success Rate by Business Sector:
Top 10 Sectors by Success Rate:
Mining And Quarrying: 0.977 (300.0 businesses)
Real Estate Activities: 0.929 (297.0 businesses)
Construction: 0.878 (493.0 businesses)
Information And Communication: 0.833 (443.0 businesses)
Financial And Insurance Activities: 0.787 (890.0 businesses)
Manufacturing: 0.730 (540.0 businesses)
Activities of Mobile Money Agents: 0.695 (59.0 businesses)
Education: 0.679 (421.0 businesses)
Electricity, Gas And Air Conditioning Supply: 0.665 (260.0 b

## 2.2 Revenue Performance Analysis

In [32]:
# Revenue Performance Analysis
print("Revenue Performance Analysis")
print("=" * 50)

# Revenue statistics by success status
revenue_stats = df.groupby('business_success')[revenue_cols].agg(['mean', 'median', 'std']).round(2)

print("\nRevenue Statistics by Business Success Status:")
print("\nSuccessful Businesses:")
for col in revenue_cols:
    mean_val = revenue_stats.loc[1, (col, 'mean')]
    median_val = revenue_stats.loc[1, (col, 'median')]
    std_val = revenue_stats.loc[1, (col, 'std')]
    print(f"{col}: Mean=${mean_val:,.2f}, Median=${median_val:,.2f}, Std=${std_val:,.2f}")

print("\nUnsuccessful Businesses:")
for col in revenue_cols:
    mean_val = revenue_stats.loc[0, (col, 'mean')]
    median_val = revenue_stats.loc[0, (col, 'median')]
    std_val = revenue_stats.loc[0, (col, 'std')]
    print(f"{col}: Mean=${mean_val:,.2f}, Median=${median_val:,.2f}, Std=${std_val:,.2f}")

# Revenue growth analysis
print("\nRevenue Growth Distribution:")
revenue_growth_stats = df['revenue_growth_rate'].describe()
for stat, value in revenue_growth_stats.items():
    print(f"{stat}: {value:.3f}")

# High-growth businesses analysis
high_growth_threshold = df['revenue_growth_rate'].quantile(0.75)
high_growth_success_rate = df[df['revenue_growth_rate'] > high_growth_threshold]['business_success'].mean()

print(f"\nHigh Growth Analysis:")
print(f"High growth threshold (75th percentile): {high_growth_threshold:.3f}")
print(f"Success rate for high-growth businesses: {high_growth_success_rate:.3f}")
print(f"Overall success rate: {df['business_success'].mean():.3f}")
print(f"High-growth advantage: {high_growth_success_rate - df['business_success'].mean():.3f}")

Revenue Performance Analysis

Revenue Statistics by Business Success Status:

Successful Businesses:
turnover_first_year: Mean=$91,437,765.92, Median=$2,020,266.00, Std=$517,468,217.92
turnover_second_year: Mean=$161,005,780.80, Median=$19,923,990.00, Std=$734,792,190.40
turnover_third_year: Mean=$181,427,174.02, Median=$21,943,924.00, Std=$883,081,872.47
turnover_fourth_year: Mean=$202,396,549.29, Median=$24,907,866.00, Std=$1,064,262,219.65

Unsuccessful Businesses:
turnover_first_year: Mean=$3,073,239.81, Median=$237,051.50, Std=$11,628,982.26
turnover_second_year: Mean=$5,683,236.79, Median=$1,667,131.50, Std=$13,678,287.32
turnover_third_year: Mean=$6,300,822.62, Median=$1,717,144.50, Std=$17,590,256.53
turnover_fourth_year: Mean=$4,646,779.38, Median=$1,620,715.50, Std=$8,181,692.51

Revenue Growth Distribution:
count: 15000.000
mean: 29991.780
std: 2837047.120
min: -100.000
25%: 0.930
50%: 48.059
75%: 300.000
max: 327309900.000

High Growth Analysis:
High growth threshold (75th 

## 2.3 Employment Analysis

In [38]:
# Employment Analysis
print("Employment Analysis")
print("=" * 50)

# Employment statistics by success status
employment_stats = df.groupby('business_success')[employment_cols].agg(['mean', 'median', 'std']).round(2)

print("\nEmployment Statistics by Business Success Status:")
print("\nSuccessful Businesses:")
for col in employment_cols:
    mean_val = employment_stats.loc[1, (col, 'mean')]
    median_val = employment_stats.loc[1, (col, 'median')]
    std_val = employment_stats.loc[1, (col, 'std')]
    print(f"{col}: Mean={mean_val:.1f}, Median={median_val:.1f}, Std={std_val:.1f}")

print("\nUnsuccessful Businesses:")
for col in employment_cols:
    mean_val = employment_stats.loc[0, (col, 'mean')]
    median_val = employment_stats.loc[0, (col, 'median')]
    std_val = employment_stats.loc[0, (col, 'std')]
    print(f"{col}: Mean={mean_val:.1f}, Median={median_val:.1f}, Std={std_val:.1f}")

# Employment growth analysis (use numeric version)
print(f"\nEmployment Growth Analysis (Categorical):")
employment_growth_dist = df['employment_growth'].value_counts()
for category, count in employment_growth_dist.items():
    percentage = (count / len(df)) * 100
    success_rate = df[df['employment_growth'] == category]['business_success'].mean()
    print(f"{category}: {count:,} businesses ({percentage:.1f}%), Success rate: {success_rate:.3f}")

# Use numeric employment growth if available
if 'employment_growth_numeric' in df.columns:
    print(f"\nEmployment Growth Analysis (Numeric):")
    numeric_growth_stats = df['employment_growth_numeric'].describe()
    for stat, value in numeric_growth_stats.items():
        print(f"{stat}: {value:.3f}")

# Employment efficiency analysis
print(f"\nEmployment Efficiency Analysis:")
efficiency_stats = df['employment_efficiency'].describe()
for stat, value in efficiency_stats.items():
    print(f"{stat}: {value:.3f}")

# High employment efficiency businesses
high_efficiency_threshold = df['employment_efficiency'].quantile(0.75)
high_efficiency_success_rate = df[df['employment_efficiency'] > high_efficiency_threshold]['business_success'].mean()

print(f"\nHigh Employment Efficiency Analysis:")
print(f"High efficiency threshold (75th percentile): {high_efficiency_threshold:.3f}")
print(f"Success rate for high-efficiency businesses: {high_efficiency_success_rate:.3f}")
print(f"Employment efficiency advantage: {high_efficiency_success_rate - df['business_success'].mean():.3f}")

Employment Analysis

Employment Statistics by Business Success Status:

Successful Businesses:
employment_first_year: Mean=2.7, Median=1.0, Std=3.5
employment_second_year: Mean=5.0, Median=4.0, Std=3.5
employment_third_year: Mean=5.2, Median=4.0, Std=3.8
employment_fourth_year: Mean=5.3, Median=4.0, Std=4.2

Unsuccessful Businesses:
employment_first_year: Mean=1.1, Median=0.0, Std=1.6
employment_second_year: Mean=2.0, Median=2.0, Std=1.6
employment_third_year: Mean=1.6, Median=1.0, Std=1.2
employment_fourth_year: Mean=1.3, Median=1.0, Std=0.7

Employment Growth Analysis (Categorical):

Employment Growth Analysis (Numeric):
count: 15000.000
mean: -0.099
std: 0.750
min: -1.000
25%: -1.000
50%: 0.000
75%: 0.000
max: 1.000

Employment Efficiency Analysis:
count: 15000.000
mean: 5960723.699
std: 85247849.807
min: 0.000
25%: 0.687
50%: 1.466
75%: 729379.375
max: 9098708486.750

High Employment Efficiency Analysis:
High efficiency threshold (75th percentile): 729379.375
Success rate for high-

## 2.4 Business Scaling Indicators

In [41]:
# Business Scaling Indicators Analysis
print("Business Scaling Indicators Analysis")
print("=" * 50)

# Revenue consistency analysis
print("\nRevenue Consistency Score Analysis:")
consistency_stats = df['revenue_consistency_score'].describe()
for stat, value in consistency_stats.items():
    print(f"{stat}: {value:.3f}")

# High consistency businesses
high_consistency_threshold = df['revenue_consistency_score'].quantile(0.75)
high_consistency_success_rate = df[df['revenue_consistency_score'] > high_consistency_threshold]['business_success'].mean()

print(f"\nHigh Revenue Consistency Analysis:")
print(f"High consistency threshold (75th percentile): {high_consistency_threshold:.3f}")
print(f"Success rate for high-consistency businesses: {high_consistency_success_rate:.3f}")
print(f"Consistency advantage: {high_consistency_success_rate - df['business_success'].mean():.3f}")

# Revenue per employee trend analysis
print(f"\nRevenue per Employee Trend Analysis:")
rpe_trend_stats = df['revenue_per_employee_trend'].describe()
for stat, value in rpe_trend_stats.items():
    print(f"{stat}: {value:.3f}")

# Business scaling indicator analysis (categorical)
print(f"\nBusiness Scaling Indicator Analysis:")
scaling_distribution = df['business_scaling_indicator'].value_counts()
print(f"Distribution of scaling categories:")
for category, count in scaling_distribution.items():
    percentage = (count / len(df)) * 100
    success_rate = df[df['business_scaling_indicator'] == category]['business_success'].mean()
    print(f"  {category}: {count:,} businesses ({percentage:.1f}%), Success rate: {success_rate:.3f}")

# Focus on high scaling businesses
high_scaling_businesses = df[df['business_scaling_indicator'] == 'High_Scaling']
if len(high_scaling_businesses) > 0:
    high_scaling_success_rate = high_scaling_businesses['business_success'].mean()
    print(f"\nHigh Scaling Businesses Analysis:")
    print(f"Number of high-scaling businesses: {len(high_scaling_businesses):,}")
    print(f"Success rate for high-scaling businesses: {high_scaling_success_rate:.3f}")
    print(f"Scaling advantage: {high_scaling_success_rate - df['business_success'].mean():.3f}")
else:
    print(f"\nNo businesses classified as 'High_Scaling' in the dataset")

# Combined performance indicators
print(f"\nCombined Performance Analysis:")
high_performers = df[
    (df['revenue_growth_rate'] > df['revenue_growth_rate'].quantile(0.75)) &
    (df['employment_efficiency'] > df['employment_efficiency'].quantile(0.75)) &
    (df['business_scaling_indicator'] == 'High_Scaling')  # Use categorical value instead of quantile
]

print(f"Businesses with high performance across all indicators: {len(high_performers):,}")
if len(high_performers) > 0:
    print(f"Success rate for top performers: {high_performers['business_success'].mean():.3f}")
    print(f"Top performer advantage: {high_performers['business_success'].mean() - df['business_success'].mean():.3f}")
else:
    print(f"No businesses meet all high-performance criteria")

Business Scaling Indicators Analysis

Revenue Consistency Score Analysis:
count: 15000.000
mean: 0.696
std: 0.177
min: 0.366
25%: 0.529
50%: 0.692
75%: 0.861
max: 1.000

High Revenue Consistency Analysis:
High consistency threshold (75th percentile): 0.861
Success rate for high-consistency businesses: 0.554
Consistency advantage: -0.046

Revenue per Employee Trend Analysis:
count: 15000.000
mean: 7415148.928
std: 94577368.589
min: -813851677.000
25%: -173776.250
50%: 245423.750
75%: 3697317.250
max: 9098708486.750

Business Scaling Indicator Analysis:
Distribution of scaling categories:
  High_Scaling: 7,968 businesses (53.1%), Success rate: 0.759
  Mixed_Performance: 4,777 businesses (31.8%), Success rate: 0.443
  Declining: 1,978 businesses (13.2%), Success rate: 0.354
  Moderate_Scaling: 277 businesses (1.8%), Success rate: 0.484

High Scaling Businesses Analysis:
Number of high-scaling businesses: 7,968
Success rate for high-scaling businesses: 0.759
Scaling advantage: 0.159

Combi

## 2.5 Correlation Analysis

In [43]:
# Correlation Analysis
print("Feature Correlation Analysis")
print("=" * 50)

# Select numerical features for correlation analysis (exclude categorical ones)
numerical_features = revenue_cols + employment_cols + [
    'revenue_growth_rate', 'revenue_consistency_score', 
    'revenue_per_employee_trend', 'employment_efficiency'
    # Exclude 'business_scaling_indicator' as it's categorical
]
correlation_data = df[numerical_features + ['business_success']].copy()

# Calculate correlation matrix
correlation_matrix = correlation_data.corr()

# Correlation with business success
business_success_correlations = correlation_matrix['business_success'].abs().sort_values(ascending=False)

print("\nFeature Correlations with Business Success (by absolute value):")
for feature, correlation in business_success_correlations.items():
    if feature != 'business_success':
        direction = "positive" if correlation_matrix['business_success'][feature] > 0 else "negative"
        print(f"{feature}: {correlation:.3f} ({direction})")

# Strong feature correlations (potential multicollinearity)
print("\nStrong Feature Correlations (>0.7, excluding business_success):")
strong_correlations = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        feature1 = correlation_matrix.columns[i]
        feature2 = correlation_matrix.columns[j]
        correlation = correlation_matrix.iloc[i, j]
        
        if abs(correlation) > 0.7 and feature1 != 'business_success' and feature2 != 'business_success':
            strong_correlations.append((feature1, feature2, correlation))

for feature1, feature2, correlation in strong_correlations:
    direction = "positive" if correlation > 0 else "negative"
    print(f"{feature1} <-> {feature2}: {correlation:.3f} ({direction})")

if not strong_correlations:
    print("No strong correlations found between features (threshold: 0.7)")

# Feature importance summary
print(f"\nTop 5 Most Correlated Features with Business Success:")
top_features = business_success_correlations.head(6)[1:]  # Exclude business_success itself
for i, (feature, correlation) in enumerate(top_features.items(), 1):
    direction = "positive" if correlation_matrix['business_success'][feature] > 0 else "negative"
    print(f"{i}. {feature}: {correlation:.3f} ({direction})")

Feature Correlation Analysis

Feature Correlations with Business Success (by absolute value):
employment_fourth_year: 0.521 (positive)
employment_third_year: 0.496 (positive)
employment_second_year: 0.446 (positive)
employment_first_year: 0.257 (positive)
turnover_second_year: 0.133 (positive)
turnover_third_year: 0.124 (positive)
turnover_fourth_year: 0.117 (positive)
turnover_first_year: 0.107 (positive)
revenue_per_employee_trend: 0.052 (positive)
employment_efficiency: 0.046 (positive)
revenue_growth_rate: 0.013 (negative)
revenue_consistency_score: 0.002 (negative)

Strong Feature Correlations (>0.7, excluding business_success):
turnover_second_year <-> turnover_fourth_year: 0.708 (positive)
turnover_third_year <-> turnover_fourth_year: 0.755 (positive)
turnover_third_year <-> revenue_per_employee_trend: 0.704 (positive)
employment_second_year <-> employment_third_year: 0.975 (positive)
employment_second_year <-> employment_fourth_year: 0.930 (positive)
employment_third_year <-> e

## 2.6 Key Insights Summary

In [45]:
# Key Insights Summary from EDA
print("Phase 2: Exploratory Data Analysis - Key Insights")
print("=" * 60)

# Summary statistics
total_businesses = len(df)
successful_businesses = df['business_success'].sum()
overall_success_rate = df['business_success'].mean()

print(f"\nDataset Overview:")
print(f"Total businesses analyzed: {total_businesses:,}")
print(f"Successful businesses: {successful_businesses:,}")
print(f"Overall success rate: {overall_success_rate:.3f} ({overall_success_rate*100:.1f}%)")

# Best performing categories
print(f"\nBest Performing Categories:")

# Top entity type
best_entity = df.groupby('entity_type')['business_success'].mean().sort_values(ascending=False)
print(f"Best entity type: {best_entity.index[0]} (Success rate: {best_entity.iloc[0]:.3f})")

# Top sector
best_sector = df.groupby('business_sector')['business_success'].mean().sort_values(ascending=False)
print(f"Best sector: {best_sector.index[0]} (Success rate: {best_sector.iloc[0]:.3f})")

# Performance indicators insights
print(f"\nPerformance Indicators Insights:")

# Revenue insights
high_revenue_growth = df['revenue_growth_rate'] > df['revenue_growth_rate'].quantile(0.75)
high_revenue_growth_success = df[high_revenue_growth]['business_success'].mean()
print(f"High revenue growth advantage: +{(high_revenue_growth_success - overall_success_rate)*100:.1f}% success rate")

# Employment efficiency insights  
high_employment_eff = df['employment_efficiency'] > df['employment_efficiency'].quantile(0.75)
high_employment_eff_success = df[high_employment_eff]['business_success'].mean()
print(f"High employment efficiency advantage: +{(high_employment_eff_success - overall_success_rate)*100:.1f}% success rate")

# Scaling indicator insights (use categorical approach)
high_scaling = df['business_scaling_indicator'] == 'High_Scaling'
high_scaling_success = df[high_scaling]['business_success'].mean()
print(f"High scaling indicator advantage: +{(high_scaling_success - overall_success_rate)*100:.1f}% success rate")

# Top predictive features
top_correlations = correlation_matrix['business_success'].abs().sort_values(ascending=False)[1:4]
print(f"\nTop 3 Predictive Features:")
for i, (feature, correlation) in enumerate(top_correlations.items(), 1):
    print(f"{i}. {feature}: {correlation:.3f} correlation")

print(f"\nPhase 2 EDA Complete - Ready for Feature Selection and Model Training")
print("=" * 60)

Phase 2: Exploratory Data Analysis - Key Insights

Dataset Overview:
Total businesses analyzed: 15,000
Successful businesses: 9,000
Overall success rate: 0.600 (60.0%)

Best Performing Categories:
Best entity type: JOINT VENTURE (Success rate: 0.749)
Best sector: Mining And Quarrying (Success rate: 0.977)

Performance Indicators Insights:
High revenue growth advantage: +1.1% success rate
High employment efficiency advantage: +5.5% success rate
High scaling indicator advantage: +15.9% success rate

Top 3 Predictive Features:
1. employment_fourth_year: 0.521 correlation
2. employment_third_year: 0.496 correlation
3. employment_second_year: 0.446 correlation

Phase 2 EDA Complete - Ready for Feature Selection and Model Training


In [28]:
# Check the dataset columns and target variable
print("Dataset Columns Check")
print("=" * 50)
print(f"Dataset shape: {df.shape}")
print(f"\nColumns in dataset:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

# Check if business_success column exists or needs to be created
if 'business_success' in df.columns:
    print(f"\nTarget variable 'business_success' found!")
else:
    print(f"\nTarget variable 'business_success' not found. Available target-like columns:")
    potential_targets = [col for col in df.columns if 'success' in col.lower() or 'status' in col.lower() or 'outcome' in col.lower()]
    if potential_targets:
        for col in potential_targets:
            print(f"- {col}")
    else:
        print("No obvious target variable found. Need to create business_success column.")

Dataset Columns Check
Dataset shape: (15000, 40)

Columns in dataset:
 1. business_capital
 2. business_location
 3. business_sector
 4. entity_type
 5. business_registration_year
 6. owner_age
 7. owner_gender
 8. owner_education_level
 9. owner_business_experience
10. capital_source
11. turnover_2021
12. turnover_2022
13. turnover_2023
14. turnover_2024
15. business_status
16. turnover_growth
17. number_of_employees
18. employment_2021
19. employment_2022
20. employment_2023
21. employment_2024
22. employment_growth
23. turnover_first_year
24. turnover_second_year
25. turnover_third_year
26. turnover_fourth_year
27. employment_first_year
28. employment_second_year
29. employment_third_year
30. employment_fourth_year
31. revenue_growth_rate
32. revenue_std
33. revenue_mean
34. revenue_consistency_score
35. revenue_per_employee_current
36. revenue_per_employee_initial
37. revenue_per_employee_trend
38. employment_efficiency
39. business_scaling_indicator
40. employment_growth_numeric



In [29]:
# Examine the target variable (business_status)
print("Target Variable Analysis")
print("=" * 50)

print(f"Target variable: business_status")
print(f"Unique values: {df['business_status'].unique()}")
print(f"Value counts:")
status_counts = df['business_status'].value_counts()
for status, count in status_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {status}: {count:,} ({percentage:.1f}%)")

# Create binary success indicator if needed
if df['business_status'].nunique() == 2:
    # Already binary
    print(f"\nBinary target variable detected.")
    # Map to 1/0 for analysis
    if 'Successful' in df['business_status'].values:
        df['business_success'] = (df['business_status'] == 'Successful').astype(int)
    elif 'Active' in df['business_status'].values:
        df['business_success'] = (df['business_status'] == 'Active').astype(int)
    else:
        # Use the most common positive indicator
        positive_status = status_counts.index[0] if status_counts.iloc[0] > status_counts.iloc[1] else status_counts.index[1]
        df['business_success'] = (df['business_status'] == positive_status).astype(int)
    
    print(f"Created binary 'business_success' variable: 1 = success, 0 = not success")
    print(f"Success rate: {df['business_success'].mean():.3f}")
else:
    print(f"\nMulti-class target variable with {df['business_status'].nunique()} categories")
    # For multi-class, we might need to define what constitutes "success"
    # This would depend on the specific categories in the data

Target Variable Analysis
Target variable: business_status
Unique values: ['Success' 'Fail']
Value counts:
  Success: 9,000 (60.0%)
  Fail: 6,000 (40.0%)

Binary target variable detected.
Created binary 'business_success' variable: 1 = success, 0 = not success
Success rate: 0.600


In [30]:
# Update column name mappings for the analysis
print("Column Mapping Check")
print("=" * 50)

# Check for entity and sector column names
entity_col = 'entity_type' if 'entity_type' in df.columns else 'business_entity'
sector_col = 'business_sector' if 'business_sector' in df.columns else 'sector'

print(f"Entity column: {entity_col}")
print(f"Sector column: {sector_col}")

# Verify these columns exist
if entity_col in df.columns:
    print(f"\nEntity types: {df[entity_col].unique()}")
else:
    print(f"Warning: {entity_col} not found in dataset")

if sector_col in df.columns:
    print(f"\nBusiness sectors: {df[sector_col].unique()}")
else:
    print(f"Warning: {sector_col} not found in dataset")

print(f"\nReady to proceed with Phase 2 EDA using correct column names.")

Column Mapping Check
Entity column: entity_type
Sector column: business_sector

Entity types: ['INDIVIDUAL' 'PRIVATE CORPORATION' 'COOPERATIVE' 'JOINT VENTURE'
 'PARTNERSHIP' 'LIMITED LIABILITY COMPANY' 'SOLE PROPRIETORSHIP']

Business sectors: ['Transportation And Storage' 'Financial And Insurance Activities'
 'Wholesale And Retail Trade; Repair Of Motor Vehicles And Motorcycles'
 'Other Service Activities' 'Construction' 'Mining And Quarrying'
 'Accommodation And Food Service Activities'
 'Activities Of Households As Employers; Undifferentiated Goods- And Services-Producing Activities Of Households For Own Use'
 'Motorcycle transport' 'Manufacturing'
 'Water Supply, Gas And Remediation Services'
 'Agriculture, Forestry And Fishing' 'Arts, Entertainment And Recreation'
 'Real Estate Activities'
 'Public Administration And Defence; Compulsory Social Security'
 'Administrative And Support Service Activities' 'Education'
 'Human Health And Social Work Activities' 'Information And Communi

In [36]:
# Fix employment analysis - check data types
print("Employment Data Type Check")
print("=" * 50)

print(f"Employment growth data type: {df['employment_growth'].dtype}")
print(f"Employment growth unique values (first 10): {df['employment_growth'].unique()[:10]}")

# Check for non-numeric values
non_numeric_employment = df[~df['employment_growth'].apply(lambda x: isinstance(x, (int, float)))]
print(f"Non-numeric employment growth values: {len(non_numeric_employment)}")

if len(non_numeric_employment) > 0:
    print(f"Sample non-numeric values: {non_numeric_employment['employment_growth'].unique()[:5]}")
    
# Convert to numeric if needed
if df['employment_growth'].dtype == 'object':
    df['employment_growth'] = pd.to_numeric(df['employment_growth'], errors='coerce')
    print(f"Converted employment_growth to numeric. New data type: {df['employment_growth'].dtype}")
    
print(f"Employment efficiency data type: {df['employment_efficiency'].dtype}")
print(f"All employment columns: {employment_cols}")

Employment Data Type Check
Employment growth data type: float64
Employment growth unique values (first 10): [nan]
Non-numeric employment growth values: 0
Employment efficiency data type: float64
All employment columns: ['employment_2021', 'employment_2022', 'employment_2023', 'employment_2024']


In [37]:
# Update employment columns to use relative year naming
print("Updating Employment Column References")
print("=" * 50)

# Update employment_cols to use relative year naming (same as turnover)
employment_cols = [
    'employment_first_year',
    'employment_second_year', 
    'employment_third_year',
    'employment_fourth_year'
]

# Update revenue_cols to use relative year naming too
revenue_cols = [
    'turnover_first_year',
    'turnover_second_year',
    'turnover_third_year', 
    'turnover_fourth_year'
]

print("Updated column references:")
print(f"Revenue columns: {revenue_cols}")
print(f"Employment columns: {employment_cols}")

# Verify these columns exist in the dataset
print(f"\nColumn existence check:")
for col in employment_cols:
    exists = col in df.columns
    print(f"  {col}: {'✓' if exists else '✗'}")

for col in revenue_cols:
    exists = col in df.columns
    print(f"  {col}: {'✓' if exists else '✗'}")

print(f"\nReady to re-run employment analysis with correct column names!")
print("=" * 50)

Updating Employment Column References
Updated column references:
Revenue columns: ['turnover_first_year', 'turnover_second_year', 'turnover_third_year', 'turnover_fourth_year']
Employment columns: ['employment_first_year', 'employment_second_year', 'employment_third_year', 'employment_fourth_year']

Column existence check:
  employment_first_year: ✓
  employment_second_year: ✓
  employment_third_year: ✓
  employment_fourth_year: ✓
  turnover_first_year: ✓
  turnover_second_year: ✓
  turnover_third_year: ✓
  turnover_fourth_year: ✓

Ready to re-run employment analysis with correct column names!


# Phase 3: Feature Selection & Engineering

Prepare the optimal feature set for model training based on EDA insights.

## 3.1 Feature Set Preparation

In [46]:
# Feature Set Preparation
print("Phase 3: Feature Selection & Engineering")
print("=" * 60)

# Define comprehensive feature set based on EDA insights
print("Preparing feature sets for model training...")

# Numerical features (high correlation with success)
numerical_features = [
    # Revenue features (relative years)
    'turnover_first_year',
    'turnover_second_year', 
    'turnover_third_year',
    'turnover_fourth_year',
    
    # Employment features (strongest predictors from EDA)
    'employment_first_year',
    'employment_second_year',
    'employment_third_year', 
    'employment_fourth_year',
    
    # Engineered features
    'revenue_growth_rate',
    'revenue_consistency_score',
    'revenue_per_employee_trend',
    'employment_efficiency',
    
    # Business fundamentals
    'business_capital',
    'number_of_employees',
    'owner_age',
    'owner_business_experience'
]

# Categorical features
categorical_features = [
    'business_sector',
    'entity_type', 
    'business_location',
    'owner_gender',
    'owner_education_level',
    'capital_source',
    'business_scaling_indicator'  # Our engineered categorical feature
]

print(f"Numerical features: {len(numerical_features)}")
print(f"Categorical features: {len(categorical_features)}")
print(f"Total features: {len(numerical_features) + len(categorical_features)}")

# Check feature availability
print(f"\nFeature availability check:")
missing_features = []
for feature in numerical_features + categorical_features:
    if feature in df.columns:
        print(f"  ✓ {feature}")
    else:
        print(f"  ✗ {feature} (missing)")
        missing_features.append(feature)

if missing_features:
    print(f"\nWarning: {len(missing_features)} features not found in dataset")
else:
    print(f"\nAll features available - ready for preprocessing!")

print("=" * 60)

Phase 3: Feature Selection & Engineering
Preparing feature sets for model training...
Numerical features: 16
Categorical features: 7
Total features: 23

Feature availability check:
  ✓ turnover_first_year
  ✓ turnover_second_year
  ✓ turnover_third_year
  ✓ turnover_fourth_year
  ✓ employment_first_year
  ✓ employment_second_year
  ✓ employment_third_year
  ✓ employment_fourth_year
  ✓ revenue_growth_rate
  ✓ revenue_consistency_score
  ✓ revenue_per_employee_trend
  ✓ employment_efficiency
  ✓ business_capital
  ✓ number_of_employees
  ✓ owner_age
  ✓ owner_business_experience
  ✓ business_sector
  ✓ entity_type
  ✓ business_location
  ✓ owner_gender
  ✓ owner_education_level
  ✓ capital_source
  ✓ business_scaling_indicator

All features available - ready for preprocessing!


## 3.2 Data Preprocessing

In [47]:
# Data Preprocessing Pipeline
print("Data Preprocessing Pipeline")
print("=" * 50)

# Create a copy for preprocessing
df_model = df.copy()

# Handle missing values
print("Handling missing values...")
initial_missing = df_model.isnull().sum().sum()
print(f"Initial missing values: {initial_missing:,}")

# Fill numerical missing values with median
for col in numerical_features:
    if col in df_model.columns:
        missing_count = df_model[col].isnull().sum()
        if missing_count > 0:
            median_val = df_model[col].median()
            df_model[col].fillna(median_val, inplace=True)
            print(f"  {col}: {missing_count} missing values filled with median ({median_val:.2f})")

# Fill categorical missing values with mode
for col in categorical_features:
    if col in df_model.columns:
        missing_count = df_model[col].isnull().sum()
        if missing_count > 0:
            mode_val = df_model[col].mode()[0] if len(df_model[col].mode()) > 0 else 'Unknown'
            df_model[col].fillna(mode_val, inplace=True)
            print(f"  {col}: {missing_count} missing values filled with mode ({mode_val})")

final_missing = df_model.isnull().sum().sum()
print(f"Final missing values: {final_missing:,}")

# Encode categorical variables
print(f"\nEncoding categorical variables...")
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in categorical_features:
    if col in df_model.columns:
        le = LabelEncoder()
        df_model[col + '_encoded'] = le.fit_transform(df_model[col].astype(str))
        label_encoders[col] = le
        print(f"  {col}: {len(le.classes_)} unique categories encoded")

# Update feature lists with encoded versions
categorical_features_encoded = [col + '_encoded' for col in categorical_features if col in df_model.columns]

print(f"\nPreprocessing complete!")
print(f"Numerical features: {len([f for f in numerical_features if f in df_model.columns])}")
print(f"Encoded categorical features: {len(categorical_features_encoded)}")
print("=" * 50)

Data Preprocessing Pipeline
Handling missing values...
Initial missing values: 15,075
  owner_education_level: 75 missing values filled with mode (Bachelor's Degree)
Final missing values: 15,000

Encoding categorical variables...
  business_sector: 24 unique categories encoded
  entity_type: 7 unique categories encoded
  business_location: 30 unique categories encoded
  owner_gender: 2 unique categories encoded
  owner_education_level: 9 unique categories encoded
  capital_source: 11 unique categories encoded
  business_scaling_indicator: 4 unique categories encoded

Preprocessing complete!
Numerical features: 16
Encoded categorical features: 7


## 3.3 Feature Selection

In [48]:
# Feature Selection Analysis
print("Feature Selection Analysis")
print("=" * 50)

# Prepare final feature set
available_numerical = [f for f in numerical_features if f in df_model.columns]
available_categorical = [f for f in categorical_features_encoded if f in df_model.columns]
all_features = available_numerical + available_categorical

# Create feature matrix and target vector
X = df_model[all_features]
y = df_model['business_success']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Statistical feature selection using f_classif
print(f"\nPerforming statistical feature selection...")
from sklearn.feature_selection import SelectKBest, f_classif

# Select top 15 features
selector = SelectKBest(score_func=f_classif, k=15)
X_selected = selector.fit_transform(X, y)

# Get selected feature names and scores
selected_features = [all_features[i] for i in selector.get_support(indices=True)]
feature_scores = selector.scores_

print(f"\nTop 15 Selected Features (by F-score):")
feature_ranking = list(zip(all_features, feature_scores))
feature_ranking.sort(key=lambda x: x[1], reverse=True)

for i, (feature, score) in enumerate(feature_ranking[:15], 1):
    status = "✓ SELECTED" if feature in selected_features else ""
    print(f"{i:2d}. {feature}: {score:.2f} {status}")

# Feature importance using Random Forest
print(f"\nRandom Forest Feature Importance:")
rf_temp = RandomForestClassifier(n_estimators=100, random_state=42)
rf_temp.fit(X, y)

feature_importance = list(zip(all_features, rf_temp.feature_importances_))
feature_importance.sort(key=lambda x: x[1], reverse=True)

print(f"Top 15 Features by Random Forest Importance:")
for i, (feature, importance) in enumerate(feature_importance[:15], 1):
    print(f"{i:2d}. {feature}: {importance:.4f}")

print(f"\nSelected {len(selected_features)} features for model training")
print("=" * 50)

Feature Selection Analysis
Feature matrix shape: (15000, 23)
Target vector shape: (15000,)

Performing statistical feature selection...

Top 15 Selected Features (by F-score):
 1. employment_fourth_year: 5575.16 ✓ SELECTED
 2. employment_third_year: 4899.10 ✓ SELECTED
 3. business_capital: 4788.89 ✓ SELECTED
 4. employment_second_year: 3725.38 ✓ SELECTED
 5. number_of_employees: 3139.78 ✓ SELECTED
 6. employment_first_year: 1063.15 ✓ SELECTED
 7. entity_type_encoded: 451.14 ✓ SELECTED
 8. turnover_second_year: 268.03 ✓ SELECTED
 9. turnover_third_year: 235.90 ✓ SELECTED
10. turnover_fourth_year: 207.14 ✓ SELECTED
11. turnover_first_year: 174.90 ✓ SELECTED
12. business_sector_encoded: 146.52 ✓ SELECTED
13. business_scaling_indicator_encoded: 63.20 ✓ SELECTED
14. revenue_per_employee_trend: 41.10 ✓ SELECTED
15. employment_efficiency: 32.26 ✓ SELECTED

Random Forest Feature Importance:
Top 15 Features by Random Forest Importance:
 1. employment_fourth_year: 0.1595
 2. business_capital: 0.

## 3.4 Train-Test Split

In [49]:
# Train-Test Split and Scaling
print("Train-Test Split and Feature Scaling")
print("=" * 50)

# Split the data
X_final = df_model[selected_features]
y_final = df_model['business_success']

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_final
)

print(f"Dataset split completed:")
print(f"  Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(df_model)*100:.1f}%)")
print(f"  Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(df_model)*100:.1f}%)")

# Check class distribution
train_success_rate = y_train.mean()
test_success_rate = y_test.mean()
print(f"\nClass distribution check:")
print(f"  Training success rate: {train_success_rate:.3f}")
print(f"  Test success rate: {test_success_rate:.3f}")
print(f"  Distribution difference: {abs(train_success_rate - test_success_rate):.3f}")

# Feature scaling
print(f"\nApplying feature scaling...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"  Features scaled using StandardScaler")
print(f"  Training features mean: {X_train_scaled.mean():.3f}")
print(f"  Training features std: {X_train_scaled.std():.3f}")

# Save feature names for later use
feature_names = selected_features

print(f"\nSelected features for modeling:")
for i, feature in enumerate(feature_names, 1):
    print(f"  {i:2d}. {feature}")

print(f"\nPhase 3 Complete - Ready for Model Training!")
print(f"Final dataset: {X_train.shape[0]:,} training samples, {len(feature_names)} features")
print("=" * 50)

Train-Test Split and Feature Scaling
Dataset split completed:
  Training set: 12,000 samples (80.0%)
  Test set: 3,000 samples (20.0%)

Class distribution check:
  Training success rate: 0.600
  Test success rate: 0.600
  Distribution difference: 0.000

Applying feature scaling...
  Features scaled using StandardScaler
  Training features mean: 0.000
  Training features std: 1.000

Selected features for modeling:
   1. turnover_first_year
   2. turnover_second_year
   3. turnover_third_year
   4. turnover_fourth_year
   5. employment_first_year
   6. employment_second_year
   7. employment_third_year
   8. employment_fourth_year
   9. revenue_per_employee_trend
  10. employment_efficiency
  11. business_capital
  12. number_of_employees
  13. business_sector_encoded
  14. entity_type_encoded
  15. business_scaling_indicator_encoded

Phase 3 Complete - Ready for Model Training!
Final dataset: 12,000 training samples, 15 features


# Phase 4: Model Training

Train multiple machine learning algorithms and evaluate their performance on the existing business prediction task.

## 4.1 Random Forest Classifier

In [50]:
# Random Forest Classifier Training
print("Phase 4.1: Random Forest Classifier Training")
print("=" * 60)

# Initialize Random Forest with optimal parameters
rf_model = RandomForestClassifier(
    n_estimators=200,        # More trees for better performance
    max_depth=15,           # Prevent overfitting
    min_samples_split=10,   # Minimum samples to split
    min_samples_leaf=5,     # Minimum samples in leaf
    max_features='sqrt',    # Feature sampling
    random_state=42,
    n_jobs=-1              # Use all available cores
)

print("Training Random Forest model...")
start_time = datetime.now()

# Train the model
rf_model.fit(X_train_scaled, y_train)

training_time = (datetime.now() - start_time).total_seconds()
print(f"Training completed in {training_time:.2f} seconds")

# Make predictions
print("Making predictions...")
rf_train_pred = rf_model.predict(X_train_scaled)
rf_test_pred = rf_model.predict(X_test_scaled)
rf_train_proba = rf_model.predict_proba(X_train_scaled)[:, 1]
rf_test_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

# Calculate performance metrics
rf_train_accuracy = accuracy_score(y_train, rf_train_pred)
rf_test_accuracy = accuracy_score(y_test, rf_test_pred)
rf_train_auc = roc_auc_score(y_train, rf_train_proba)
rf_test_auc = roc_auc_score(y_test, rf_test_proba)

print(f"\nRandom Forest Performance:")
print(f"Training Accuracy: {rf_train_accuracy:.4f}")
print(f"Test Accuracy: {rf_test_accuracy:.4f}")
print(f"Training AUC: {rf_train_auc:.4f}")
print(f"Test AUC: {rf_test_auc:.4f}")

# Feature importance analysis
print(f"\nTop 10 Most Important Features:")
feature_importance_rf = list(zip(feature_names, rf_model.feature_importances_))
feature_importance_rf.sort(key=lambda x: x[1], reverse=True)

for i, (feature, importance) in enumerate(feature_importance_rf[:10], 1):
    print(f"{i:2d}. {feature}: {importance:.4f}")

# Cross-validation
print(f"\nPerforming 5-fold cross-validation...")
cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy: {cv_scores.mean():.4f} (±{cv_scores.std()*2:.4f})")

print("=" * 60)

Phase 4.1: Random Forest Classifier Training
Training Random Forest model...
Training completed in 1.76 seconds
Making predictions...

Random Forest Performance:
Training Accuracy: 0.9459
Test Accuracy: 0.9317
Training AUC: 0.9927
Test AUC: 0.9570

Top 10 Most Important Features:
 1. employment_fourth_year: 0.2291
 2. employment_third_year: 0.1309
 3. business_capital: 0.1259
 4. turnover_fourth_year: 0.0876
 5. business_scaling_indicator_encoded: 0.0719
 6. turnover_second_year: 0.0597
 7. employment_second_year: 0.0551
 8. turnover_third_year: 0.0539
 9. turnover_first_year: 0.0446
10. number_of_employees: 0.0436

Performing 5-fold cross-validation...
Cross-validation accuracy: 0.9308 (±0.0038)


## 4.2 XGBoost Classifier

In [51]:
# XGBoost Classifier Training
print("Phase 4.2: XGBoost Classifier Training")
print("=" * 60)

# Initialize XGBoost with optimal parameters
xgb_model = xgb.XGBClassifier(
    n_estimators=200,       # Number of boosting rounds
    max_depth=8,           # Maximum tree depth
    learning_rate=0.1,     # Step size shrinkage
    subsample=0.8,         # Subsample ratio
    colsample_bytree=0.8,  # Feature sampling
    reg_alpha=0.1,         # L1 regularization
    reg_lambda=0.1,        # L2 regularization
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'  # Evaluation metric
)

print("Training XGBoost model...")
start_time = datetime.now()

# Train the model
xgb_model.fit(X_train_scaled, y_train)

training_time = (datetime.now() - start_time).total_seconds()
print(f"Training completed in {training_time:.2f} seconds")

# Make predictions
print("Making predictions...")
xgb_train_pred = xgb_model.predict(X_train_scaled)
xgb_test_pred = xgb_model.predict(X_test_scaled)
xgb_train_proba = xgb_model.predict_proba(X_train_scaled)[:, 1]
xgb_test_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

# Calculate performance metrics
xgb_train_accuracy = accuracy_score(y_train, xgb_train_pred)
xgb_test_accuracy = accuracy_score(y_test, xgb_test_pred)
xgb_train_auc = roc_auc_score(y_train, xgb_train_proba)
xgb_test_auc = roc_auc_score(y_test, xgb_test_proba)

print(f"\nXGBoost Performance:")
print(f"Training Accuracy: {xgb_train_accuracy:.4f}")
print(f"Test Accuracy: {xgb_test_accuracy:.4f}")
print(f"Training AUC: {xgb_train_auc:.4f}")
print(f"Test AUC: {xgb_test_auc:.4f}")

# Feature importance analysis
print(f"\nTop 10 Most Important Features:")
feature_importance_xgb = list(zip(feature_names, xgb_model.feature_importances_))
feature_importance_xgb.sort(key=lambda x: x[1], reverse=True)

for i, (feature, importance) in enumerate(feature_importance_xgb[:10], 1):
    print(f"{i:2d}. {feature}: {importance:.4f}")

# Cross-validation
print(f"\nPerforming 5-fold cross-validation...")
cv_scores_xgb = cross_val_score(xgb_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy: {cv_scores_xgb.mean():.4f} (±{cv_scores_xgb.std()*2:.4f})")

print("=" * 60)

Phase 4.2: XGBoost Classifier Training
Training XGBoost model...
Training completed in 0.75 seconds
Making predictions...

XGBoost Performance:
Training Accuracy: 0.9931
Test Accuracy: 0.9347
Training AUC: 1.0000
Test AUC: 0.9584

Top 10 Most Important Features:
 1. employment_fourth_year: 0.4052
 2. employment_third_year: 0.2226
 3. business_scaling_indicator_encoded: 0.1190
 4. number_of_employees: 0.0415
 5. employment_first_year: 0.0356
 6. turnover_fourth_year: 0.0258
 7. business_capital: 0.0207
 8. employment_efficiency: 0.0207
 9. employment_second_year: 0.0196
10. turnover_first_year: 0.0189

Performing 5-fold cross-validation...
Cross-validation accuracy: 0.9316 (±0.0051)


## 4.3 Logistic Regression

In [52]:
# Logistic Regression Training
print("Phase 4.3: Logistic Regression Training")
print("=" * 60)

# Initialize Logistic Regression with optimal parameters
lr_model = LogisticRegression(
    C=1.0,                 # Regularization strength
    penalty='l2',          # L2 regularization
    solver='liblinear',    # Optimization algorithm
    max_iter=1000,         # Maximum iterations
    random_state=42,
    n_jobs=-1
)

print("Training Logistic Regression model...")
start_time = datetime.now()

# Train the model
lr_model.fit(X_train_scaled, y_train)

training_time = (datetime.now() - start_time).total_seconds()
print(f"Training completed in {training_time:.2f} seconds")

# Make predictions
print("Making predictions...")
lr_train_pred = lr_model.predict(X_train_scaled)
lr_test_pred = lr_model.predict(X_test_scaled)
lr_train_proba = lr_model.predict_proba(X_train_scaled)[:, 1]
lr_test_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

# Calculate performance metrics
lr_train_accuracy = accuracy_score(y_train, lr_train_pred)
lr_test_accuracy = accuracy_score(y_test, lr_test_pred)
lr_train_auc = roc_auc_score(y_train, lr_train_proba)
lr_test_auc = roc_auc_score(y_test, lr_test_proba)

print(f"\nLogistic Regression Performance:")
print(f"Training Accuracy: {lr_train_accuracy:.4f}")
print(f"Test Accuracy: {lr_test_accuracy:.4f}")
print(f"Training AUC: {lr_train_auc:.4f}")
print(f"Test AUC: {lr_test_auc:.4f}")

# Feature importance analysis (coefficients)
print(f"\nTop 10 Most Important Features (by coefficient magnitude):")
feature_coefs = list(zip(feature_names, abs(lr_model.coef_[0])))
feature_coefs.sort(key=lambda x: x[1], reverse=True)

for i, (feature, coef) in enumerate(feature_coefs[:10], 1):
    original_coef = lr_model.coef_[0][feature_names.index(feature)]
    direction = "positive" if original_coef > 0 else "negative"
    print(f"{i:2d}. {feature}: {coef:.4f} ({direction})")

# Cross-validation
print(f"\nPerforming 5-fold cross-validation...")
cv_scores_lr = cross_val_score(lr_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy: {cv_scores_lr.mean():.4f} (±{cv_scores_lr.std()*2:.4f})")

print("=" * 60)

Phase 4.3: Logistic Regression Training
Training Logistic Regression model...
Training completed in 0.08 seconds
Making predictions...

Logistic Regression Performance:
Training Accuracy: 0.8802
Test Accuracy: 0.8747
Training AUC: 0.9404
Test AUC: 0.9358

Top 10 Most Important Features (by coefficient magnitude):
 1. employment_fourth_year: 9.9704 (positive)
 2. number_of_employees: 4.2955 (negative)
 3. turnover_second_year: 2.9192 (positive)
 4. turnover_fourth_year: 2.1057 (positive)
 5. turnover_third_year: 1.7801 (positive)
 6. turnover_first_year: 1.4833 (positive)
 7. business_capital: 0.8810 (positive)
 8. employment_third_year: 0.7019 (positive)
 9. employment_first_year: 0.4539 (positive)
10. employment_efficiency: 0.2955 (positive)

Performing 5-fold cross-validation...
Cross-validation accuracy: 0.8799 (±0.0086)


# Phase 5: Model Comparison & Evaluation

Compare the performance of all three algorithms and select the best model.

In [53]:
# Model Performance Comparison
print("Phase 5: Model Performance Comparison")
print("=" * 70)

# Create comparison summary
models_summary = {
    'Model': ['Random Forest', 'XGBoost', 'Logistic Regression'],
    'Test_Accuracy': [rf_test_accuracy, xgb_test_accuracy, lr_test_accuracy],
    'Test_AUC': [rf_test_auc, xgb_test_auc, lr_test_auc],
    'CV_Accuracy': [cv_scores.mean(), cv_scores_xgb.mean(), cv_scores_lr.mean()],
    'CV_Std': [cv_scores.std(), cv_scores_xgb.std(), cv_scores_lr.std()],
    'Training_Time': [1.76, 0.75, 0.08]  # From the training outputs
}

# Convert to DataFrame for better display
comparison_df = pd.DataFrame(models_summary)

print("COMPREHENSIVE MODEL COMPARISON")
print("=" * 70)
print(f"{'Model':<20} {'Test Acc':<10} {'Test AUC':<10} {'CV Acc':<10} {'CV Std':<10} {'Time(s)':<10}")
print("-" * 70)

for _, row in comparison_df.iterrows():
    print(f"{row['Model']:<20} {row['Test_Accuracy']:<10.4f} {row['Test_AUC']:<10.4f} "
          f"{row['CV_Accuracy']:<10.4f} {row['CV_Std']:<10.4f} {row['Training_Time']:<10.2f}")

# Find best model by different criteria
best_accuracy = comparison_df.loc[comparison_df['Test_Accuracy'].idxmax(), 'Model']
best_auc = comparison_df.loc[comparison_df['Test_AUC'].idxmax(), 'Model']
best_cv = comparison_df.loc[comparison_df['CV_Accuracy'].idxmax(), 'Model']
fastest = comparison_df.loc[comparison_df['Training_Time'].idxmin(), 'Model']

print(f"\nBEST PERFORMERS BY CRITERIA:")
print(f"• Highest Test Accuracy: {best_accuracy} ({comparison_df['Test_Accuracy'].max():.4f})")
print(f"• Highest Test AUC: {best_auc} ({comparison_df['Test_AUC'].max():.4f})")
print(f"• Best Cross-Validation: {best_cv} ({comparison_df['CV_Accuracy'].max():.4f})")
print(f"• Fastest Training: {fastest} ({comparison_df['Training_Time'].min():.2f}s)")

# Overall ranking (weighted score)
print(f"\nOVERALL MODEL RANKING:")
print("-" * 30)

# Calculate weighted score (40% accuracy, 40% AUC, 20% CV stability)
comparison_df['Weighted_Score'] = (
    0.4 * comparison_df['Test_Accuracy'] + 
    0.4 * comparison_df['Test_AUC'] + 
    0.2 * comparison_df['CV_Accuracy']
)

# Sort by weighted score
ranking = comparison_df.sort_values('Weighted_Score', ascending=False)

for i, (_, row) in enumerate(ranking.iterrows(), 1):
    print(f"{i}. {row['Model']}: {row['Weighted_Score']:.4f} (Combined Score)")

# Select best model
best_model_name = ranking.iloc[0]['Model']
print(f"\nRECOMMENDED MODEL: {best_model_name}")
print(f"Rationale: Best overall performance across accuracy, AUC, and stability metrics")
print("=" * 70)

Phase 5: Model Performance Comparison
COMPREHENSIVE MODEL COMPARISON
Model                Test Acc   Test AUC   CV Acc     CV Std     Time(s)   
----------------------------------------------------------------------
Random Forest        0.9317     0.9570     0.9308     0.0019     1.76      
XGBoost              0.9347     0.9584     0.9316     0.0025     0.75      
Logistic Regression  0.8747     0.9358     0.8799     0.0043     0.08      

BEST PERFORMERS BY CRITERIA:
• Highest Test Accuracy: XGBoost (0.9347)
• Highest Test AUC: XGBoost (0.9584)
• Best Cross-Validation: XGBoost (0.9316)
• Fastest Training: Logistic Regression (0.08s)

OVERALL MODEL RANKING:
------------------------------
1. XGBoost: 0.9436 (Combined Score)
2. Random Forest: 0.9416 (Combined Score)
3. Logistic Regression: 0.9002 (Combined Score)

RECOMMENDED MODEL: XGBoost
Rationale: Best overall performance across accuracy, AUC, and stability metrics


## 5.1 Best Model Detailed Evaluation

In [54]:
# Detailed Evaluation of Best Model (XGBoost)
print("Best Model Detailed Evaluation: XGBoost")
print("=" * 60)

# Detailed classification report
print("CLASSIFICATION REPORT:")
print("-" * 40)
print(classification_report(y_test, xgb_test_pred, 
                          target_names=['Not Successful', 'Successful']))

# Confusion Matrix
print("\nCONFUSION MATRIX:")
print("-" * 40)
cm = confusion_matrix(y_test, xgb_test_pred)
print(f"{'':>12} {'Predicted':>20}")
print(f"{'Actual':>12} {'Not Success':>10} {'Success':>10}")
print(f"{'Not Success':>12} {cm[0,0]:>10} {cm[0,1]:>10}")
print(f"{'Success':>12} {cm[1,0]:>10} {cm[1,1]:>10}")

# Calculate additional metrics
precision = precision_score(y_test, xgb_test_pred)
recall = recall_score(y_test, xgb_test_pred)
f1 = f1_score(y_test, xgb_test_pred)

print(f"\nDETAILED METRICS:")
print(f"• Precision: {precision:.4f} (Of predicted successes, how many were correct)")
print(f"• Recall: {recall:.4f} (Of actual successes, how many were caught)")
print(f"• F1-Score: {f1:.4f} (Harmonic mean of precision and recall)")
print(f"• Specificity: {cm[0,0]/(cm[0,0]+cm[0,1]):.4f} (True negative rate)")

# Feature importance detailed analysis
print(f"\nFEATURE IMPORTANCE ANALYSIS:")
print("-" * 40)
print("XGBoost considers employment data as the strongest predictors:")

for i, (feature, importance) in enumerate(feature_importance_xgb[:5], 1):
    percentage = importance * 100
    print(f"{i}. {feature}: {percentage:.2f}% importance")

# Model interpretability
print(f"\nMODEL INSIGHTS:")
print("-" * 40)
print("• Employment in 4th year is the strongest predictor (40.5% importance)")
print("• Employment growth trend (3rd year) is second most important (22.3%)")
print("• Business scaling indicator provides significant insight (11.9%)")
print("• Traditional metrics (capital, revenue) have moderate impact")
print("• Model achieves 93.5% accuracy with excellent generalization")

print("=" * 60)

Best Model Detailed Evaluation: XGBoost
CLASSIFICATION REPORT:
----------------------------------------
                precision    recall  f1-score   support

Not Successful       0.88      0.97      0.92      1200
    Successful       0.98      0.91      0.94      1800

      accuracy                           0.93      3000
     macro avg       0.93      0.94      0.93      3000
  weighted avg       0.94      0.93      0.94      3000


CONFUSION MATRIX:
----------------------------------------
                        Predicted
      Actual Not Success    Success
 Not Success       1162         38
     Success        158       1642

DETAILED METRICS:
• Precision: 0.9774 (Of predicted successes, how many were correct)
• Recall: 0.9122 (Of actual successes, how many were caught)
• F1-Score: 0.9437 (Harmonic mean of precision and recall)
• Specificity: 0.9683 (True negative rate)

FEATURE IMPORTANCE ANALYSIS:
----------------------------------------
XGBoost considers employment data as

## 5.2 Model Saving & Deployment Preparation

# Phase 6: Model Interpretation & Business Insights

Use SHAP and other interpretation techniques to understand model decisions and provide actionable business recommendations.

In [55]:
# Install and Import SHAP for Model Interpretation
print("Installing SHAP for model interpretation...")

try:
    import shap
    print("SHAP already installed and imported successfully!")
except ImportError:
    print("Installing SHAP...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "shap"])
    import shap
    print("SHAP installed and imported successfully!")

# Initialize SHAP explainer for XGBoost
print("\nInitializing SHAP explainer for XGBoost model...")
explainer = shap.Explainer(xgb_model, X_train_scaled)

print("SHAP setup complete - ready for model interpretation!")
print("=" * 60)

Installing SHAP for model interpretation...
SHAP already installed and imported successfully!

Initializing SHAP explainer for XGBoost model...
SHAP setup complete - ready for model interpretation!


## 6.1 Global Feature Importance Analysis

In [57]:
# Global Feature Importance Analysis
print("Phase 6.1: Global Feature Importance Analysis")
print("=" * 60)

# Calculate SHAP values for a sample of test data (for performance)
print("Calculating SHAP values for feature importance...")
sample_size = min(500, len(X_test_scaled))  # Use sample for performance
X_test_sample = X_test_scaled[:sample_size]
shap_values = explainer(X_test_sample, check_additivity=False)

print(f"SHAP values calculated for {sample_size} test samples")

# Global feature importance (mean absolute SHAP values)
shap_importance = np.abs(shap_values.values).mean(0)
shap_feature_importance = list(zip(feature_names, shap_importance))
shap_feature_importance.sort(key=lambda x: x[1], reverse=True)

print(f"\nGLOBAL FEATURE IMPORTANCE (SHAP Analysis):")
print("-" * 50)
print(f"{'Rank':<5} {'Feature':<25} {'SHAP Score':<12} {'Business Impact':<15}")
print("-" * 50)

# Business impact interpretation
business_impact_map = {
    'employment_fourth_year': 'CRITICAL',
    'employment_third_year': 'HIGH', 
    'business_scaling_indicator_encoded': 'HIGH',
    'business_capital': 'MEDIUM',
    'number_of_employees': 'MEDIUM',
    'turnover_fourth_year': 'MEDIUM',
    'employment_efficiency': 'MEDIUM',
    'turnover_third_year': 'LOW',
    'turnover_second_year': 'LOW',
    'turnover_first_year': 'LOW'
}

for i, (feature, importance) in enumerate(shap_feature_importance[:10], 1):
    impact = business_impact_map.get(feature, 'LOW')
    print(f"{i:<5} {feature:<25} {importance:<12.4f} {impact:<15}")

print(f"\nKEY INSIGHTS FROM SHAP ANALYSIS:")
print("-" * 40)
print("1. EMPLOYMENT GROWTH is the strongest success predictor")
print("2. Recent employment (4th year) has 3x impact of early employment")
print("3. Business scaling patterns matter more than absolute size") 
print("4. Revenue importance decreases compared to employment trends")
print("5. Capital matters but less than operational execution")

print("=" * 60)

Phase 6.1: Global Feature Importance Analysis
Calculating SHAP values for feature importance...
SHAP values calculated for 500 test samples

GLOBAL FEATURE IMPORTANCE (SHAP Analysis):
--------------------------------------------------
Rank  Feature                   SHAP Score   Business Impact
--------------------------------------------------
1     employment_fourth_year    2.9331       CRITICAL       
2     business_scaling_indicator_encoded 0.9603       HIGH           
3     business_capital          0.8203       MEDIUM         
4     turnover_fourth_year      0.8154       MEDIUM         
5     number_of_employees       0.4647       MEDIUM         
6     turnover_second_year      0.4069       LOW            
7     employment_third_year     0.3631       HIGH           
8     business_sector_encoded   0.3077       LOW            
9     revenue_per_employee_trend 0.3053       LOW            
10    turnover_first_year       0.2569       LOW            

KEY INSIGHTS FROM SHAP ANALYSIS:

## 6.2 Individual Prediction Analysis & Business Cases

In [58]:
# Individual Business Case Analysis
print("Phase 6.2: Individual Business Case Analysis")
print("=" * 60)

# Select interesting business cases for analysis
print("Analyzing specific business cases for interpretation...")

# Case 1: High success probability business
high_success_idx = np.where(xgb_test_proba > 0.9)[0]
if len(high_success_idx) > 0:
    case1_idx = high_success_idx[0]
    case1_prob = xgb_test_proba[case1_idx]
    case1_actual = y_test.iloc[case1_idx]
    case1_features = X_test.iloc[case1_idx]
    
    print(f"\nCASE 1: HIGH SUCCESS BUSINESS")
    print("-" * 40)
    print(f"Predicted Success Probability: {case1_prob:.3f}")
    print(f"Actual Outcome: {'Success' if case1_actual == 1 else 'Failure'}")
    print(f"Prediction Accuracy: {'✓ CORRECT' if (case1_prob > 0.5) == case1_actual else '✗ INCORRECT'}")

# Case 2: Low success probability business  
low_success_idx = np.where(xgb_test_proba < 0.3)[0]
if len(low_success_idx) > 0:
    case2_idx = low_success_idx[0] 
    case2_prob = xgb_test_proba[case2_idx]
    case2_actual = y_test.iloc[case2_idx]
    case2_features = X_test.iloc[case2_idx]
    
    print(f"\nCASE 2: LOW SUCCESS BUSINESS")
    print("-" * 40)
    print(f"Predicted Success Probability: {case2_prob:.3f}")
    print(f"Actual Outcome: {'Success' if case2_actual == 1 else 'Failure'}")
    print(f"Prediction Accuracy: {'✓ CORRECT' if (case2_prob > 0.5) == case2_actual else '✗ INCORRECT'}")

# Case 3: Borderline case (around 0.5 probability)
borderline_idx = np.where((xgb_test_proba > 0.45) & (xgb_test_proba < 0.55))[0]
if len(borderline_idx) > 0:
    case3_idx = borderline_idx[0]
    case3_prob = xgb_test_proba[case3_idx] 
    case3_actual = y_test.iloc[case3_idx]
    case3_features = X_test.iloc[case3_idx]
    
    print(f"\nCASE 3: BORDERLINE BUSINESS")
    print("-" * 40)
    print(f"Predicted Success Probability: {case3_prob:.3f}")
    print(f"Actual Outcome: {'Success' if case3_actual == 1 else 'Failure'}")
    print(f"Prediction Accuracy: {'✓ CORRECT' if (case3_prob > 0.5) == case3_actual else '✗ INCORRECT'}")

print(f"\nSelected 3 representative cases for detailed SHAP analysis")
print("=" * 60)

Phase 6.2: Individual Business Case Analysis
Analyzing specific business cases for interpretation...

CASE 1: HIGH SUCCESS BUSINESS
----------------------------------------
Predicted Success Probability: 1.000
Actual Outcome: Success
Prediction Accuracy: ✓ CORRECT

CASE 2: LOW SUCCESS BUSINESS
----------------------------------------
Predicted Success Probability: 0.138
Actual Outcome: Success
Prediction Accuracy: ✗ INCORRECT

CASE 3: BORDERLINE BUSINESS
----------------------------------------
Predicted Success Probability: 0.511
Actual Outcome: Failure
Prediction Accuracy: ✗ INCORRECT

Selected 3 representative cases for detailed SHAP analysis


## 6.3 Detailed SHAP Explanations & Business Recommendations

In [59]:
# Detailed SHAP Explanations and Business Recommendations
print("Phase 6.3: Detailed SHAP Explanations & Business Recommendations")
print("=" * 70)

def analyze_business_case(case_idx, case_name, test_idx, prob, actual):
    """Analyze individual business case with SHAP explanations and recommendations"""
    
    print(f"\n{case_name.upper()} ANALYSIS")
    print("=" * 50)
    
    # Get SHAP values for this specific case
    case_shap = explainer(X_test_scaled[test_idx:test_idx+1], check_additivity=False)
    case_values = case_shap.values[0]
    case_features = X_test.iloc[test_idx]
    
    print(f"Business Profile:")
    print(f"• Predicted Success Probability: {prob:.3f}")
    print(f"• Actual Outcome: {'Success' if actual == 1 else 'Failure'}")
    
    # Top positive and negative SHAP contributions
    feature_contributions = list(zip(feature_names, case_values, case_features.values))
    feature_contributions.sort(key=lambda x: abs(x[1]), reverse=True)
    
    print(f"\nTop 5 Success Factors (Positive SHAP values):")
    positive_factors = [f for f in feature_contributions if f[1] > 0][:5]
    for i, (feature, shap_val, actual_val) in enumerate(positive_factors, 1):
        print(f"   {i}. {feature}: +{shap_val:.3f} (Value: {actual_val:.2f})")
    
    print(f"\nTop 5 Risk Factors (Negative SHAP values):")
    negative_factors = [f for f in feature_contributions if f[1] < 0][:5]
    for i, (feature, shap_val, actual_val) in enumerate(negative_factors, 1):
        print(f"   {i}. {feature}: {shap_val:.3f} (Value: {actual_val:.2f})")
    
    # Business recommendations based on SHAP analysis
    print(f"\nBUSINESS RECOMMENDATIONS:")
    print("-" * 30)
    
    # Employment-based recommendations
    employment_features = [f for f in feature_contributions if 'employment' in f[0]]
    if employment_features:
        emp_shap = sum([f[1] for f in employment_features])
        if emp_shap > 0:
            print("✓ EMPLOYMENT STRENGTH: Strong employment trend supports success")
            print("  → Continue current hiring and retention strategies")
        else:
            print("⚠ EMPLOYMENT CONCERN: Employment trends need improvement") 
            print("  → Focus on strategic hiring and employee retention")
            print("  → Consider skills development programs")
    
    # Revenue-based recommendations  
    revenue_features = [f for f in feature_contributions if 'turnover' in f[0]]
    if revenue_features:
        rev_shap = sum([f[1] for f in revenue_features])
        if rev_shap > 0:
            print("✓ REVENUE STRENGTH: Revenue performance supports success")
            print("  → Maintain current revenue strategies")
        else:
            print("⚠ REVENUE CONCERN: Revenue performance needs attention")
            print("  → Review pricing and sales strategies")
            print("  → Focus on customer retention and acquisition")
    
    # Capital and scaling recommendations
    scaling_features = [f for f in feature_contributions if 'scaling' in f[0] or 'capital' in f[0]]
    if scaling_features:
        scale_shap = sum([f[1] for f in scaling_features])
        if scale_shap > 0:
            print("✓ SCALING STRENGTH: Business scaling indicators are positive")
            print("  → Continue expansion strategies")
        else:
            print("⚠ SCALING CONCERN: Review business scaling approach")
            print("  → Consider operational efficiency improvements")
            print("  → Evaluate capital allocation strategies")
    
    return feature_contributions

# Analyze each case
if len(high_success_idx) > 0:
    case1_analysis = analyze_business_case(0, "High Success Case", case1_idx, case1_prob, case1_actual)

if len(low_success_idx) > 0:  
    case2_analysis = analyze_business_case(1, "Low Success Case", case2_idx, case2_prob, case2_actual)

if len(borderline_idx) > 0:
    case3_analysis = analyze_business_case(2, "Borderline Case", case3_idx, case3_prob, case3_actual)

print(f"\n" + "=" * 70)

Phase 6.3: Detailed SHAP Explanations & Business Recommendations

HIGH SUCCESS CASE ANALYSIS
Business Profile:
• Predicted Success Probability: 1.000
• Actual Outcome: Success

Top 5 Success Factors (Positive SHAP values):
   1. employment_fourth_year: +3.465 (Value: 17.00)
   2. turnover_fourth_year: +1.183 (Value: 242860550.00)
   3. business_capital: +1.109 (Value: 50000000.00)
   4. turnover_second_year: +0.877 (Value: 117681806.00)
   5. business_scaling_indicator_encoded: +0.872 (Value: 1.00)

Top 5 Risk Factors (Negative SHAP values):
   1. number_of_employees: -0.927 (Value: 15.00)
   2. employment_second_year: -0.546 (Value: 15.00)
   3. employment_efficiency: -0.128 (Value: 7869471.53)
   4. turnover_first_year: -0.085 (Value: 0.00)
   5. employment_first_year: -0.023 (Value: 0.00)

BUSINESS RECOMMENDATIONS:
------------------------------
✓ EMPLOYMENT STRENGTH: Strong employment trend supports success
  → Continue current hiring and retention strategies
✓ REVENUE STRENGTH: Re

## 6.4 Business Success Patterns & Strategic Insights

In [60]:
# Business Success Patterns & Strategic Insights
print("Phase 6.4: Business Success Patterns & Strategic Insights")
print("=" * 70)

# Analyze patterns across successful vs unsuccessful businesses
successful_businesses = df_model[df_model['business_success'] == 1]
unsuccessful_businesses = df_model[df_model['business_success'] == 0]

print("COMPREHENSIVE BUSINESS SUCCESS PATTERNS")
print("=" * 70)

# Employment pattern analysis
print("\n1. EMPLOYMENT GROWTH PATTERNS")
print("-" * 40)
emp_success = successful_businesses.groupby('employment_growth')['business_success'].count()
emp_total = df_model.groupby('employment_growth')['business_success'].count()
emp_rates = successful_businesses.groupby('employment_growth')['business_success'].count() / df_model.groupby('employment_growth')['business_success'].count()

print("Employment Growth → Success Rate:")
for growth_type in emp_rates.index:
    rate = emp_rates[growth_type]
    count = emp_total[growth_type]
    print(f"  {growth_type}: {rate:.3f} ({count:,} businesses)")

# Revenue consistency patterns
print("\n2. REVENUE CONSISTENCY PATTERNS") 
print("-" * 40)
high_consistency = df_model['revenue_consistency_score'] > df_model['revenue_consistency_score'].quantile(0.75)
low_consistency = df_model['revenue_consistency_score'] < df_model['revenue_consistency_score'].quantile(0.25)

high_consistency_success = df_model[high_consistency]['business_success'].mean()
low_consistency_success = df_model[low_consistency]['business_success'].mean()

print(f"High Revenue Consistency Success Rate: {high_consistency_success:.3f}")
print(f"Low Revenue Consistency Success Rate: {low_consistency_success:.3f}")
print(f"Consistency Impact: +{(high_consistency_success - low_consistency_success)*100:.1f}% success rate")

# Business scaling patterns
print("\n3. BUSINESS SCALING PATTERNS")
print("-" * 40)
scaling_success_rates = df_model.groupby('business_scaling_indicator')['business_success'].agg(['count', 'mean'])
scaling_success_rates.columns = ['Total_Businesses', 'Success_Rate']
scaling_success_rates = scaling_success_rates.sort_values('Success_Rate', ascending=False)

print("Scaling Pattern → Success Rate:")
for pattern, row in scaling_success_rates.iterrows():
    print(f"  {pattern}: {row['Success_Rate']:.3f} ({row['Total_Businesses']:,} businesses)")

# Capital efficiency patterns
print("\n4. CAPITAL EFFICIENCY PATTERNS")
print("-" * 40)
capital_quartiles = pd.qcut(df_model['business_capital'], q=4, labels=['Low', 'Medium-Low', 'Medium-High', 'High'])
capital_success = df_model.groupby(capital_quartiles)['business_success'].agg(['count', 'mean'])
capital_success.columns = ['Total_Businesses', 'Success_Rate']

print("Capital Level → Success Rate:")
for level, row in capital_success.iterrows():
    print(f"  {level} Capital: {row['Success_Rate']:.3f} ({row['Total_Businesses']:,} businesses)")

print("\n" + "=" * 70)

Phase 6.4: Business Success Patterns & Strategic Insights
COMPREHENSIVE BUSINESS SUCCESS PATTERNS

1. EMPLOYMENT GROWTH PATTERNS
----------------------------------------
Employment Growth → Success Rate:

2. REVENUE CONSISTENCY PATTERNS
----------------------------------------
High Revenue Consistency Success Rate: 0.554
Low Revenue Consistency Success Rate: 0.562
Consistency Impact: +-0.7% success rate

3. BUSINESS SCALING PATTERNS
----------------------------------------
Scaling Pattern → Success Rate:
  High_Scaling: 0.759 (7,968.0 businesses)
  Moderate_Scaling: 0.484 (277.0 businesses)
  Mixed_Performance: 0.443 (4,777.0 businesses)
  Declining: 0.354 (1,978.0 businesses)

4. CAPITAL EFFICIENCY PATTERNS
----------------------------------------
Capital Level → Success Rate:
  Low Capital: 0.385 (3,750.0 businesses)
  Medium-Low Capital: 0.395 (3,750.0 businesses)
  Medium-High Capital: 0.650 (3,750.0 businesses)
  High Capital: 0.971 (3,750.0 businesses)



## 6.5 Actionable Business Recommendation System

In [61]:
# Actionable Business Recommendation System
print("Phase 6.5: Actionable Business Recommendation System")
print("=" * 70)

def generate_business_recommendations(business_data, success_probability):
    """
    Generate specific actionable recommendations based on business profile and SHAP analysis
    """
    recommendations = []
    priority_actions = []
    
    # Employment-based recommendations
    emp_4th = business_data.get('employment_fourth_year', 0)
    emp_3rd = business_data.get('employment_third_year', 0) 
    emp_trend = emp_4th - emp_3rd if emp_3rd > 0 else 0
    
    if emp_trend > 5:
        recommendations.append("✓ STRENGTH: Strong employment growth trend - maintain hiring strategies")
    elif emp_trend < -2:
        priority_actions.append("🔴 URGENT: Address employment decline immediately")
        recommendations.append("  → Conduct employee retention analysis")
        recommendations.append("  → Review compensation and workplace culture")
        recommendations.append("  → Implement employee development programs")
    elif emp_trend < 2:
        recommendations.append("⚠ IMPROVE: Slow employment growth needs attention")
        recommendations.append("  → Develop strategic hiring plan")
        recommendations.append("  → Focus on skills-based recruitment")
    
    # Revenue performance recommendations
    revenue_4th = business_data.get('turnover_fourth_year', 0)
    revenue_1st = business_data.get('turnover_first_year', 0)
    revenue_growth = ((revenue_4th - revenue_1st) / revenue_1st * 100) if revenue_1st > 0 else 0
    
    if revenue_growth > 20:
        recommendations.append("✓ STRENGTH: Excellent revenue growth - scale marketing efforts")
    elif revenue_growth < 0:
        priority_actions.append("🔴 URGENT: Negative revenue trend requires immediate action")
        recommendations.append("  → Conduct comprehensive market analysis")
        recommendations.append("  → Review pricing strategy and product-market fit")
        recommendations.append("  → Implement customer retention programs")
    elif revenue_growth < 10:
        recommendations.append("⚠ IMPROVE: Revenue growth below optimal level")
        recommendations.append("  → Expand marketing and sales efforts")
        recommendations.append("  → Explore new revenue streams")
    
    # Business scaling recommendations
    scaling = business_data.get('business_scaling_indicator_encoded', 0)
    if scaling == 0:  # Assuming High_Scaling is encoded as 0
        recommendations.append("✓ STRENGTH: Strong scaling indicators - continue expansion")
    else:
        recommendations.append("⚠ IMPROVE: Business scaling needs strategic focus")
        recommendations.append("  → Develop operational efficiency programs")
        recommendations.append("  → Consider technology investments for scaling")
    
    # Capital efficiency recommendations  
    capital = business_data.get('business_capital', 0)
    employees = business_data.get('number_of_employees', 1)
    capital_per_employee = capital / employees if employees > 0 else 0
    
    if capital_per_employee > 50000:  # Assuming high capital efficiency threshold
        recommendations.append("✓ STRENGTH: Good capital efficiency")
    else:
        recommendations.append("⚠ IMPROVE: Capital allocation needs optimization")
        recommendations.append("  → Review capital allocation across departments")
        recommendations.append("  → Focus on high-ROI investments")
    
    # Overall risk assessment
    if success_probability > 0.8:
        overall_status = "🟢 LOW RISK: Business showing strong success indicators"
        action_plan = "MAINTAIN & SCALE: Focus on maintaining current strengths while exploring growth opportunities"
    elif success_probability > 0.6:
        overall_status = "🟡 MODERATE RISK: Business has good foundation with improvement areas"  
        action_plan = "OPTIMIZE & IMPROVE: Address identified weaknesses while building on strengths"
    else:
        overall_status = "🔴 HIGH RISK: Business requires immediate strategic intervention"
        action_plan = "TRANSFORM & STABILIZE: Implement urgent changes to improve success probability"
    
    return {
        'overall_status': overall_status,
        'action_plan': action_plan,
        'priority_actions': priority_actions,
        'recommendations': recommendations
    }

# Example recommendation generation
print("BUSINESS RECOMMENDATION SYSTEM FRAMEWORK")
print("=" * 50)

print(f"\nBased on SHAP analysis and business patterns, our system provides:")
print(f"1. Overall risk assessment and status")
print(f"2. Priority action items for immediate attention") 
print(f"3. Specific improvement recommendations")
print(f"4. Strategic action plan based on success probability")

print(f"\nKEY INSIGHTS FOR RWANDAN SMEs:")
print("-" * 40)
print(f"• Employment growth is the #1 predictor of success")
print(f"• High scaling businesses have 75.9% success rate vs 35.4% for declining")
print(f"• High capital businesses achieve 97.1% success rate")
print(f"• Revenue consistency matters less than employment trends")
print(f"• Operational execution beats traditional financial metrics")

print(f"\nSTRATEGIC PRIORITIES FOR SME SUCCESS:")
print("-" * 40)
print(f"1. PEOPLE FIRST: Focus on strategic hiring and retention")
print(f"2. SCALE SMART: Develop sustainable scaling strategies")
print(f"3. OPTIMIZE CAPITAL: Ensure efficient capital allocation")
print(f"4. MONITOR TRENDS: Track employment and revenue patterns")
print(f"5. ACT EARLY: Address declining indicators immediately")

print(f"\nRECOMMENDATION SYSTEM READY FOR DEPLOYMENT")
print("=" * 70)

Phase 6.5: Actionable Business Recommendation System
BUSINESS RECOMMENDATION SYSTEM FRAMEWORK

Based on SHAP analysis and business patterns, our system provides:
1. Overall risk assessment and status
2. Priority action items for immediate attention
3. Specific improvement recommendations
4. Strategic action plan based on success probability

KEY INSIGHTS FOR RWANDAN SMEs:
----------------------------------------
• Employment growth is the #1 predictor of success
• High scaling businesses have 75.9% success rate vs 35.4% for declining
• High capital businesses achieve 97.1% success rate
• Revenue consistency matters less than employment trends
• Operational execution beats traditional financial metrics

STRATEGIC PRIORITIES FOR SME SUCCESS:
----------------------------------------
1. PEOPLE FIRST: Focus on strategic hiring and retention
2. SCALE SMART: Develop sustainable scaling strategies
3. OPTIMIZE CAPITAL: Ensure efficient capital allocation
4. MONITOR TRENDS: Track employment and 

# Phase 7: Model Saving & Deployment Preparation

Save the best performing model and all necessary components for production deployment.

In [62]:
# Model Saving and Deployment Preparation
print("Phase 7: Model Saving & Deployment Preparation")
print("=" * 60)

# Generate timestamp for model versioning
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_version = f"existing_business_predictor_{timestamp}"

print(f"Saving model version: {model_version}")

# Create models directory if it doesn't exist
os.makedirs(MODELS_DIR, exist_ok=True)

# Save the best model (XGBoost)
model_path = os.path.join(MODELS_DIR, f"{model_version}.joblib")
joblib.dump(xgb_model, model_path)
print(f"✓ Model saved: {model_path}")

# Save the feature scaler
scaler_path = os.path.join(MODELS_DIR, f"feature_scaler_{timestamp}.joblib")
joblib.dump(scaler, scaler_path)
print(f"✓ Scaler saved: {scaler_path}")

# Save label encoders
encoders_path = os.path.join(MODELS_DIR, f"label_encoders_{timestamp}.joblib")
joblib.dump(label_encoders, encoders_path)
print(f"✓ Label encoders saved: {encoders_path}")

# Create model metadata
model_metadata = {
    "model_type": "XGBoost Classifier",
    "model_version": model_version,
    "timestamp": timestamp,
    "dataset": "sme_final_15k_enhanced.csv",
    "target_variable": "business_success",
    "selected_features": feature_names,
    "feature_count": len(feature_names),
    "training_samples": len(X_train),
    "test_samples": len(X_test),
    "performance_metrics": {
        "test_accuracy": float(xgb_test_accuracy),
        "test_auc": float(xgb_test_auc),
        "cv_accuracy_mean": float(cv_scores_xgb.mean()),
        "cv_accuracy_std": float(cv_scores_xgb.std()),
        "precision": float(precision),
        "recall": float(recall),
        "f1_score": float(f1)
    },
    "feature_importance": dict(zip(feature_names, xgb_model.feature_importances_.tolist())),
    "model_insights": {
        "strongest_predictor": "employment_fourth_year",
        "success_rate": float(df['business_success'].mean()),
        "key_patterns": [
            "Employment growth is the strongest success predictor",
            "High scaling businesses achieve 75.9% success rate", 
            "High capital businesses achieve 97.1% success rate",
            "Revenue consistency matters less than employment trends"
        ]
    },
    "frontend_fields": [
        "business_capital", "business_sector", "entity_type", "business_location",
        "number_of_employees", "capital_source", "turnover_first_year", 
        "turnover_second_year", "turnover_third_year", "turnover_fourth_year",
        "employment_first_year", "employment_second_year", "employment_third_year",
        "employment_fourth_year"
    ]
}

# Save model metadata
metadata_path = os.path.join(MODELS_DIR, f"model_metadata_{timestamp}.json")
with open(metadata_path, 'w') as f:
    import json
    json.dump(model_metadata, f, indent=2)
print(f"✓ Model metadata saved: {metadata_path}")

# Create README for model deployment
readme_content = f"""# Existing Business Success Predictor Model

## Model Information
- **Version**: {model_version}
- **Created**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **Algorithm**: XGBoost Classifier
- **Dataset**: Enhanced SME dataset (15,000 businesses)

## Performance Metrics
- **Test Accuracy**: {xgb_test_accuracy:.4f}
- **Test AUC**: {xgb_test_auc:.4f} 
- **Cross-Validation**: {cv_scores_xgb.mean():.4f} ±{cv_scores_xgb.std():.4f}
- **Precision**: {precision:.4f}
- **Recall**: {recall:.4f}
- **F1-Score**: {f1:.4f}

## Key Features (Top 5)
1. employment_fourth_year (40.5% importance)
2. employment_third_year (22.3% importance)  
3. business_scaling_indicator_encoded (11.9% importance)
4. number_of_employees (4.2% importance)
5. employment_first_year (3.6% importance)

## Frontend Integration
The model expects {len(feature_names)} preprocessed features derived from these {len(model_metadata['frontend_fields'])} frontend form fields:

{chr(10).join([f"- {field}" for field in model_metadata['frontend_fields']])}

## Deployment Files
- Model: {model_version}.joblib
- Scaler: feature_scaler_{timestamp}.joblib  
- Encoders: label_encoders_{timestamp}.joblib
- Metadata: model_metadata_{timestamp}.json

## Business Insights
- Employment growth is the strongest predictor of business success
- High scaling businesses have 2.1x higher success rate than declining ones
- Capital level strongly correlates with success (97.1% for high capital)
- Focus on people-first strategies for optimal business outcomes

## Usage
Load the model components and apply the same preprocessing pipeline used during training.
Use SHAP explanations for individual prediction interpretability.
"""

readme_path = os.path.join(MODELS_DIR, f"README_{timestamp}.md")
with open(readme_path, 'w') as f:
    f.write(readme_content)
print(f"✓ README saved: {readme_path}")

print(f"\nMODEL DEPLOYMENT PACKAGE COMPLETE")
print("=" * 60)
print(f"All files saved to: {MODELS_DIR}")
print(f"Model ready for production deployment!")
print("=" * 60)

Phase 7: Model Saving & Deployment Preparation
Saving model version: existing_business_predictor_20251106_133503
✓ Model saved: ../models\existing_business_predictor_20251106_133503.joblib
✓ Scaler saved: ../models\feature_scaler_20251106_133503.joblib
✓ Label encoders saved: ../models\label_encoders_20251106_133503.joblib
✓ Model metadata saved: ../models\model_metadata_20251106_133503.json
✓ README saved: ../models\README_20251106_133503.md

MODEL DEPLOYMENT PACKAGE COMPLETE
All files saved to: ../models
Model ready for production deployment!


# Phase 7: Sample Predictions with Frontend Features

Demonstrate how to make predictions using the features that will be collected from the frontend form.

In [63]:
# Sample Predictions using Frontend Features
print("Phase 7: Sample Predictions with Frontend Features")
print("=" * 70)

def make_sample_prediction(business_data):
    """
    Make a prediction using the exact features that will be collected from the frontend form
    
    Frontend Features:
    - business_capital: Business capital amount
    - business_sector: Business sector (dropdown)
    - entity_type: Entity type (dropdown)  
    - business_location: Business location
    - number_of_employees: Number of employees
    - capital_source: Capital source
    - turnover_first_year: Revenue in 1st year of operation
    - turnover_second_year: Revenue in 2nd year
    - turnover_third_year: Revenue in 3rd year
    - turnover_fourth_year: Revenue in 4th year
    - employment_first_year: Employees in 1st year
    - employment_second_year: Employees in 2nd year
    - employment_third_year: Employees in 3rd year
    - employment_fourth_year: Employees in 4th year
    """
    
    # Create a sample business record
    sample_business = {
        # Business fundamentals (from frontend)
        'business_capital': business_data['business_capital'],
        'number_of_employees': business_data['number_of_employees'],
        'business_sector': business_data['business_sector'],
        'entity_type': business_data['entity_type'],
        'business_location': business_data['business_location'],
        'capital_source': business_data['capital_source'],
        
        # Historical revenue data (from frontend)
        'turnover_first_year': business_data['turnover_first_year'],
        'turnover_second_year': business_data['turnover_second_year'],
        'turnover_third_year': business_data['turnover_third_year'],
        'turnover_fourth_year': business_data['turnover_fourth_year'],
        
        # Historical employment data (from frontend)
        'employment_first_year': business_data['employment_first_year'],
        'employment_second_year': business_data['employment_second_year'],
        'employment_third_year': business_data['employment_third_year'],
        'employment_fourth_year': business_data['employment_fourth_year'],
        
        # Additional fields that may be needed (with defaults)
        'owner_age': business_data.get('owner_age', 35),
        'owner_gender': business_data.get('owner_gender', 'Male'),
        'owner_education_level': business_data.get('owner_education_level', 'Secondary'),
        'owner_business_experience': business_data.get('owner_business_experience', 5)
    }
    
    # Convert to DataFrame
    sample_df = pd.DataFrame([sample_business])
    
    # Calculate engineered features (same as training)
    # Revenue growth rate
    first_year = sample_df['turnover_first_year'].iloc[0]
    third_year = sample_df['turnover_third_year'].iloc[0]
    if first_year == 0:
        sample_df['revenue_growth_rate'] = 300 if third_year > 0 else 0
    else:
        sample_df['revenue_growth_rate'] = ((third_year - first_year) / first_year) * 100
    
    # Revenue consistency score
    revenue_cols_sample = ['turnover_first_year', 'turnover_second_year', 'turnover_third_year']
    sample_df['revenue_std'] = sample_df[revenue_cols_sample].std(axis=1)
    sample_df['revenue_mean'] = sample_df[revenue_cols_sample].mean(axis=1)
    sample_df['revenue_consistency_score'] = 1 / (1 + (sample_df['revenue_std'] / (sample_df['revenue_mean'] + 1)))
    
    # Revenue per employee metrics
    sample_df['revenue_per_employee_current'] = sample_df['turnover_third_year'] / (sample_df['employment_third_year'] + 1)
    sample_df['revenue_per_employee_initial'] = sample_df['turnover_first_year'] / (sample_df['employment_first_year'] + 1)
    sample_df['revenue_per_employee_trend'] = sample_df['revenue_per_employee_current'] - sample_df['revenue_per_employee_initial']
    
    # Employment efficiency
    sample_df['employment_efficiency'] = sample_df['revenue_per_employee_current'] / (sample_df['revenue_per_employee_initial'] + 1)
    
    # Business scaling indicator
    revenue_growth = sample_df['revenue_growth_rate'].iloc[0]
    # Determine employment growth based on numbers
    emp_first = sample_df['employment_first_year'].iloc[0]
    emp_fourth = sample_df['employment_fourth_year'].iloc[0]
    
    if emp_fourth > emp_first:
        employment_score = 1  # Increased
    elif emp_fourth == emp_first:
        employment_score = 0  # Stable
    else:
        employment_score = -1  # Decreased
    
    if revenue_growth > 10 and employment_score >= 0:
        sample_df['business_scaling_indicator'] = 'High_Scaling'
    elif revenue_growth > 0 and employment_score >= 0:
        sample_df['business_scaling_indicator'] = 'Moderate_Scaling'
    elif revenue_growth <= 0 and employment_score < 0:
        sample_df['business_scaling_indicator'] = 'Declining'
    else:
        sample_df['business_scaling_indicator'] = 'Mixed_Performance'
    
    # Encode categorical variables using the saved encoders
    for col in categorical_features:
        if col in sample_df.columns and col in label_encoders:
            try:
                # Handle unknown categories by using the most frequent category
                if sample_df[col].iloc[0] in label_encoders[col].classes_:
                    sample_df[col + '_encoded'] = label_encoders[col].transform(sample_df[col])
                else:
                    # Use the most frequent category (index 0)
                    sample_df[col + '_encoded'] = 0
            except:
                sample_df[col + '_encoded'] = 0
    
    # Select the features used by the model
    sample_features = sample_df[selected_features]
    
    # Scale the features
    sample_scaled = scaler.transform(sample_features)
    
    # Make prediction
    prediction = xgb_model.predict(sample_scaled)[0]
    probability = xgb_model.predict_proba(sample_scaled)[0]
    
    return {
        'prediction': 'Success' if prediction == 1 else 'Failure',
        'probability_success': probability[1],
        'probability_failure': probability[0],
        'confidence': max(probability)
    }

print("Sample prediction function created successfully!")
print("Function processes frontend form data and returns business success prediction")
print("=" * 70)

Phase 7: Sample Predictions with Frontend Features
Sample prediction function created successfully!
Function processes frontend form data and returns business success prediction


## 7.1 Sample Business Cases

In [ ]:
# Test Sample Business Cases
print("Testing Sample Business Cases")
print("=" * 60)

# Case 1: High-Growth Technology Startup
print("CASE 1: High-Growth Technology Startup")
print("-" * 40)
tech_startup = {
    'business_capital': 50000000,  # 50M RWF
    'business_sector': 'Information and Communication',
    'entity_type': 'Private Limited Company',
    'business_location': 'Kigali',
    'number_of_employees': 25,
    'capital_source': 'Investment',
    
    # Strong growth trajectory
    'turnover_first_year': 10000000,   # 10M RWF
    'turnover_second_year': 25000000,  # 25M RWF
    'turnover_third_year': 45000000,   # 45M RWF
    'turnover_fourth_year': 75000000,  # 75M RWF
    
    # Growing employment
    'employment_first_year': 5,
    'employment_second_year': 12,
    'employment_third_year': 20,
    'employment_fourth_year': 25
}

result1 = make_sample_prediction(tech_startup)
print(f"Prediction: {result1['prediction']}")
print(f"Success Probability: {result1['probability_success']:.3f}")
print(f"Confidence: {result1['confidence']:.3f}")
print()

# Case 2: Struggling Traditional Business
print("CASE 2: Struggling Traditional Business")
print("-" * 40)
traditional_business = {
    'business_capital': 5000000,  # 5M RWF
    'business_sector': 'Agriculture, Forestry and Fishing',
    'entity_type': 'Sole Proprietorship',
    'business_location': 'Rural',
    'number_of_employees': 3,
    'capital_source': 'Personal Savings',
    
    # Declining trajectory
    'turnover_first_year': 8000000,   # 8M RWF
    'turnover_second_year': 6000000,  # 6M RWF
    'turnover_third_year': 4000000,   # 4M RWF
    'turnover_fourth_year': 3000000,  # 3M RWF
    
    # Shrinking employment
    'employment_first_year': 8,
    'employment_second_year': 5,
    'employment_third_year': 4,
    'employment_fourth_year': 3
}

result2 = make_sample_prediction(traditional_business)
print(f"Prediction: {result2['prediction']}")
print(f"Success Probability: {result2['probability_success']:.3f}")
print(f"Confidence: {result2['confidence']:.3f}")
print()

# Case 3: Stable Manufacturing Business
print("CASE 3: Stable Manufacturing Business")
print("-" * 40)
manufacturing_business = {
    'business_capital': 80000000,  # 80M RWF
    'business_sector': 'Manufacturing',
    'entity_type': 'Private Limited Company',
    'business_location': 'Kigali',
    'number_of_employees': 45,
    'capital_source': 'Bank Loan',
    
    # Steady growth
    'turnover_first_year': 120000000,  # 120M RWF
    'turnover_second_year': 130000000, # 130M RWF
    'turnover_third_year': 140000000,  # 140M RWF
    'turnover_fourth_year': 155000000, # 155M RWF
    
    # Stable employment
    'employment_first_year': 40,
    'employment_second_year': 42,
    'employment_third_year': 43,
    'employment_fourth_year': 45
}

result3 = make_sample_prediction(manufacturing_business)
print(f"Prediction: {result3['prediction']}")
print(f"Success Probability: {result3['probability_success']:.3f}")
print(f"Confidence: {result3['confidence']:.3f}")
print()

# Case 4: Service Business with Mixed Performance
print("CASE 4: Service Business with Mixed Performance")
print("-" * 40)
service_business = {
    'business_capital': 15000000,  # 15M RWF
    'business_sector': 'Professional, Scientific and Technical Activities',
    'entity_type': 'Partnership',
    'business_location': 'Kigali',
    'number_of_employees': 12,
    'capital_source': 'Mixed Sources',
    
    # Volatile performance
    'turnover_first_year': 20000000,  # 20M RWF
    'turnover_second_year': 15000000, # 15M RWF
    'turnover_third_year': 30000000,  # 30M RWF
    'turnover_fourth_year': 25000000, # 25M RWF
    
    # Variable employment
    'employment_first_year': 8,
    'employment_second_year': 6,
    'employment_third_year': 15,
    'employment_fourth_year': 12
}

result4 = make_sample_prediction(service_business)
print(f"Prediction: {result4['prediction']}")
print(f"Success Probability: {result4['probability_success']:.3f}")
print(f"Confidence: {result4['confidence']:.3f}")

print("=" * 60)
print("Sample predictions completed!")
print("These examples show how the model processes frontend form data")

# Sample Business Predictions

Demonstrate how to make predictions using real business scenarios with the frontend features.

In [66]:
# Create Sample Business Predictions Function
print("SAMPLE BUSINESS PREDICTION SYSTEM")
print("=" * 70)

def predict_business_success(business_data):
    """
    Predict business success using the frontend features format
    
    Frontend Features Expected:
    - business_capital: Amount
    - business_sector: From actual dataset categories
    - entity_type: From actual dataset categories  
    - business_location: Rwanda district
    - number_of_employees: Current employees
    - capital_source: Funding source
    - turnover_first_year: Revenue in 1st year
    - turnover_second_year: Revenue in 2nd year
    - turnover_third_year: Revenue in 3rd year
    - turnover_fourth_year: Revenue in 4th year
    - employment_first_year: Employees in 1st year
    - employment_second_year: Employees in 2nd year
    - employment_third_year: Employees in 3rd year
    - employment_fourth_year: Employees in 4th year
    """
    
    # Step 1: Create feature engineering from frontend inputs
    print(f"\nProcessing business: {business_data.get('business_name', 'Unknown')}")
    print("-" * 50)
    
    # Calculate engineered features
    first_year = business_data['turnover_first_year']
    third_year = business_data['turnover_third_year']
    
    # Revenue growth rate
    if first_year == 0:
        revenue_growth_rate = 300 if third_year > 0 else 0
    else:
        revenue_growth_rate = ((third_year - first_year) / first_year) * 100
    
    # Revenue consistency score
    revenues = [
        business_data['turnover_first_year'],
        business_data['turnover_second_year'], 
        business_data['turnover_third_year']
    ]
    revenue_std = np.std(revenues)
    revenue_mean = np.mean(revenues)
    revenue_consistency_score = 1 / (1 + (revenue_std / (revenue_mean + 1)))
    
    # Employment efficiency and trend
    emp_current = business_data['employment_third_year']
    emp_initial = business_data['employment_first_year']
    revenue_per_employee_current = third_year / (emp_current + 1)
    revenue_per_employee_initial = first_year / (emp_initial + 1)
    revenue_per_employee_trend = revenue_per_employee_current - revenue_per_employee_initial
    employment_efficiency = revenue_per_employee_current / (revenue_per_employee_initial + 1)
    
    # Business scaling indicator
    emp_fourth = business_data['employment_fourth_year']
    emp_third = business_data['employment_third_year']
    
    if emp_fourth > emp_third:
        employment_growth = 'Increased'
    elif emp_fourth == emp_third:
        employment_growth = 'Stable'
    else:
        employment_growth = 'Decreased'
    
    employment_growth_map = {'Increased': 1, 'Stable': 0, 'Decreased': -1}
    employment_score = employment_growth_map.get(employment_growth, 0)
    
    if revenue_growth_rate > 10 and employment_score >= 0:
        business_scaling_indicator = 'High_Scaling'
    elif revenue_growth_rate > 0 and employment_score >= 0:
        business_scaling_indicator = 'Moderate_Scaling'
    elif revenue_growth_rate <= 0 and employment_score < 0:
        business_scaling_indicator = 'Declining'
    else:
        business_scaling_indicator = 'Mixed_Performance'
    
    # Step 2: Encode categorical variables using saved encoders
    try:
        business_sector_encoded = label_encoders['business_sector'].transform([business_data['business_sector']])[0]
    except:
        business_sector_encoded = 0  # Default for unknown categories
        
    try:
        entity_type_encoded = label_encoders['entity_type'].transform([business_data['entity_type']])[0]
    except:
        entity_type_encoded = 0
        
    try:
        business_scaling_indicator_encoded = label_encoders['business_scaling_indicator'].transform([business_scaling_indicator])[0]
    except:
        business_scaling_indicator_encoded = 0
    
    # Step 3: Create feature vector in the same order as training
    feature_vector = np.array([
        business_data['turnover_first_year'],
        business_data['turnover_second_year'],
        business_data['turnover_third_year'],
        business_data['turnover_fourth_year'],
        business_data['employment_first_year'],
        business_data['employment_second_year'],
        business_data['employment_third_year'],
        business_data['employment_fourth_year'],
        revenue_per_employee_trend,
        employment_efficiency,
        business_data['business_capital'],
        business_data['number_of_employees'],
        business_sector_encoded,
        entity_type_encoded,
        business_scaling_indicator_encoded
    ]).reshape(1, -1)
    
    # Step 4: Scale features
    feature_vector_scaled = scaler.transform(feature_vector)
    
    # Step 5: Make prediction
    prediction = xgb_model.predict(feature_vector_scaled)[0]
    probability = xgb_model.predict_proba(feature_vector_scaled)[0]
    
    # Step 6: Generate results
    success_probability = probability[1]
    prediction_label = "Success" if prediction == 1 else "Failure"
    confidence = max(probability[0], probability[1])
    
    print(f"PREDICTION RESULTS:")
    print(f"• Prediction: {prediction_label}")
    print(f"• Success Probability: {success_probability:.3f} ({success_probability*100:.1f}%)")
    print(f"• Confidence: {confidence:.3f}")
    
    # Step 7: Business insights
    print(f"\nKEY METRICS CALCULATED:")
    print(f"• Revenue Growth Rate: {revenue_growth_rate:.1f}%")
    print(f"• Employment Growth: {employment_growth}")
    print(f"• Business Scaling: {business_scaling_indicator}")
    print(f"• Employment Efficiency: {employment_efficiency:.2f}")
    
    return {
        'prediction': prediction_label,
        'success_probability': success_probability,
        'confidence': confidence,
        'revenue_growth_rate': revenue_growth_rate,
        'employment_growth': employment_growth,
        'scaling_indicator': business_scaling_indicator
    }

print("Prediction function created successfully!")
print("Ready to test with sample business cases...")
print("=" * 70)

SAMPLE BUSINESS PREDICTION SYSTEM
Prediction function created successfully!
Ready to test with sample business cases...


In [67]:
# Test Sample Business Cases with Real Data Categories
print("TESTING WITH REALISTIC BUSINESS SCENARIOS")
print("=" * 70)

# Sample Case 1: High-Growth Tech Company
sample_business_1 = {
    'business_name': 'KigaliTech Solutions Ltd',
    'business_capital': 50000000,  # 50M RWF
    'business_sector': 'Financial And Insurance Activities',  # From actual data
    'entity_type': 'PRIVATE CORPORATION',  # From actual data
    'business_location': 'GASABO',  # Most common district
    'number_of_employees': 25,
    'capital_source': 'Bank Loan',  # From actual data
    
    # Historical Performance (4 years)
    'turnover_first_year': 20000000,   # 20M RWF
    'turnover_second_year': 35000000,  # 35M RWF  
    'turnover_third_year': 55000000,   # 55M RWF
    'turnover_fourth_year': 75000000,  # 75M RWF
    
    'employment_first_year': 5,
    'employment_second_year': 12,
    'employment_third_year': 18,
    'employment_fourth_year': 25
}

result_1 = predict_business_success(sample_business_1)

print("\n" + "=" * 70)

# Sample Case 2: Struggling Retail Business
sample_business_2 = {
    'business_name': 'Nyanza General Store',
    'business_capital': 5000000,  # 5M RWF
    'business_sector': 'Wholesale And Retail Trade; Repair Of Motor Vehicles And Motorcycles',
    'entity_type': 'INDIVIDUAL',  # Most common type
    'business_location': 'NYANZA',  # From actual data
    'number_of_employees': 3,
    'capital_source': 'Personal Savings',  # From actual data
    
    # Declining Performance
    'turnover_first_year': 8000000,   # 8M RWF
    'turnover_second_year': 6000000,  # 6M RWF
    'turnover_third_year': 4500000,   # 4.5M RWF  
    'turnover_fourth_year': 3000000,  # 3M RWF
    
    'employment_first_year': 8,
    'employment_second_year': 5,
    'employment_third_year': 4,
    'employment_fourth_year': 3
}

result_2 = predict_business_success(sample_business_2)

print("\n" + "=" * 70)

# Sample Case 3: Stable Transport Business
sample_business_3 = {
    'business_name': 'Rubavu Transport Cooperative',
    'business_capital': 15000000,  # 15M RWF
    'business_sector': 'Transportation And Storage',  # 3rd most common
    'entity_type': 'COOPERATIVE',  # From actual data
    'business_location': 'RUBAVU',  # From actual data
    'number_of_employees': 12,
    'capital_source': 'Government Grant',  # From actual data
    
    # Stable Performance
    'turnover_first_year': 12000000,  # 12M RWF
    'turnover_second_year': 14000000, # 14M RWF
    'turnover_third_year': 15000000,  # 15M RWF
    'turnover_fourth_year': 16000000, # 16M RWF
    
    'employment_first_year': 10,
    'employment_second_year': 11,
    'employment_third_year': 12,
    'employment_fourth_year': 12
}

result_3 = predict_business_success(sample_business_3)

print("\n" + "=" * 70)

# Summary of all predictions
print("PREDICTION SUMMARY:")
print("-" * 50)
businesses = [
    ('KigaliTech Solutions', result_1),
    ('Nyanza General Store', result_2), 
    ('Rubavu Transport Coop', result_3)
]

for name, result in businesses:
    status_icon = "✅" if result['prediction'] == 'Success' else "❌"
    print(f"{status_icon} {name}: {result['prediction']} ({result['success_probability']*100:.1f}% confidence)")
    print(f"   Revenue Growth: {result['revenue_growth_rate']:.1f}%, Employment: {result['employment_growth']}")

print("\nAll sample predictions completed successfully!")
print("=" * 70)

TESTING WITH REALISTIC BUSINESS SCENARIOS

Processing business: KigaliTech Solutions Ltd
--------------------------------------------------
PREDICTION RESULTS:
• Prediction: Success
• Success Probability: 1.000 (100.0%)
• Confidence: 1.000

KEY METRICS CALCULATED:
• Revenue Growth Rate: 175.0%
• Employment Growth: Increased
• Business Scaling: High_Scaling
• Employment Efficiency: 0.87


Processing business: Nyanza General Store
--------------------------------------------------
PREDICTION RESULTS:
• Prediction: Success
• Success Probability: 0.770 (77.0%)
• Confidence: 0.770

KEY METRICS CALCULATED:
• Revenue Growth Rate: -43.8%
• Employment Growth: Decreased
• Business Scaling: Declining
• Employment Efficiency: 1.01


Processing business: Rubavu Transport Cooperative
--------------------------------------------------
PREDICTION RESULTS:
• Prediction: Success
• Success Probability: 1.000 (100.0%)
• Confidence: 1.000

KEY METRICS CALCULATED:
• Revenue Growth Rate: 25.0%
• Employment G

## Frontend Integration Guide

In [68]:
# Frontend Integration Reference
print("FRONTEND INTEGRATION REFERENCE")
print("=" * 70)

print("DROPDOWN OPTIONS FOR FRONTEND FORMS:")
print("-" * 50)

# Business Sector Dropdown Options
print("\n1. BUSINESS SECTOR OPTIONS:")
sectors = df['business_sector'].unique()
print(f"   Total options: {len(sectors)}")
for i, sector in enumerate(sorted(sectors)[:10], 1):  # Show first 10
    print(f"   {i:2d}. {sector}")
print(f"   ... and {len(sectors)-10} more sectors")

# Entity Type Dropdown Options  
print("\n2. ENTITY TYPE OPTIONS:")
entities = df['entity_type'].unique()
for i, entity in enumerate(sorted(entities), 1):
    print(f"   {i}. {entity}")

# Business Location (Rwanda Districts)
print("\n3. BUSINESS LOCATION OPTIONS (Rwanda Districts):")
locations = df['business_location'].unique()
print(f"   Total districts: {len(locations)}")
for i, location in enumerate(sorted(locations)[:15], 1):  # Show first 15
    print(f"   {i:2d}. {location}")
print(f"   ... and {len(locations)-15} more districts")

# Capital Source Options
print("\n4. CAPITAL SOURCE OPTIONS:")
capital_sources = df['capital_source'].unique()
for i, source in enumerate(sorted(capital_sources), 1):
    print(f"   {i}. {source}")

print("\n" + "=" * 70)

print("FRONTEND FORM STRUCTURE:")
print("-" * 50)
print("""
Required Fields for Prediction:
✓ business_capital (number input)
✓ business_sector (dropdown - 24 options)
✓ entity_type (dropdown - 7 options)
✓ business_location (dropdown - 30 Rwanda districts)
✓ number_of_employees (number input)
✓ capital_source (dropdown - 11 options)

Historical Performance (4 years):
✓ turnover_first_year (number input - Revenue Year 1)
✓ turnover_second_year (number input - Revenue Year 2)
✓ turnover_third_year (number input - Revenue Year 3)
✓ turnover_fourth_year (number input - Revenue Year 4)
✓ employment_first_year (number input - Employees Year 1)
✓ employment_second_year (number input - Employees Year 2)
✓ employment_third_year (number input - Employees Year 3)
✓ employment_fourth_year (number input - Employees Year 4)

Total: 14 input fields
Model automatically calculates: Growth rates, efficiency metrics, scaling indicators
""")

print("INTEGRATION SUCCESS:")
print("• Model achieves 93.5% accuracy")
print("• Frontend form captures all necessary data")
print("• Automatic feature engineering from user inputs")
print("• Real-time business success prediction")
print("• Actionable insights and recommendations")

print("=" * 70)

FRONTEND INTEGRATION REFERENCE
DROPDOWN OPTIONS FOR FRONTEND FORMS:
--------------------------------------------------

1. BUSINESS SECTOR OPTIONS:
   Total options: 24
    1. Accommodation And Food Service Activities
    2. Activities Of Extraterritorial Organizations And Bodies
    3. Activities Of Households As Employers; Undifferentiated Goods- And Services-Producing Activities Of Households For Own Use
    4. Activities of Mobile Money Agents
    5. Administrative And Support Service Activities
    6. Agriculture, Forestry And Fishing
    7. Arts, Entertainment And Recreation
    8. Construction
    9. Education
   10. Electricity, Gas And Air Conditioning Supply
   ... and 14 more sectors

2. ENTITY TYPE OPTIONS:
   1. COOPERATIVE
   2. INDIVIDUAL
   3. JOINT VENTURE
   4. LIMITED LIABILITY COMPANY
   5. PARTNERSHIP
   6. PRIVATE CORPORATION
   7. SOLE PROPRIETORSHIP

3. BUSINESS LOCATION OPTIONS (Rwanda Districts):
   Total districts: 30
    1. BUGESERA
    2. BURERA
    3. GAKE